In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd '/content/drive/My Drive/sequicity-master/'

/content/drive/My Drive/sequicity-master


In [ ]:
# import jovian

<IPython.core.display.Javascript object>

In [ ]:
# jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Please enter your API key ( from https://jovian.ml/ ):
API KEY: ········
[jovian] Uploading notebook..
[jovian] Capturing environment..


[jovian] Error: Failed to read Anaconda environment using command: "conda env export -n base --no-builds"


[jovian] Committed successfully! https://jovian.ml/suprabhatkr/seq2seq-b652a


'https://jovian.ml/suprabhatkr/seq2seq-b652a'

In [4]:
import keras
import numpy as np
from keras.models import Sequential,Model
from keras.layers import Dense,Dropout,LSTM,Input,Bidirectional
from sklearn.model_selection import cross_val_score 
from nltk.tokenize import word_tokenize,sent_tokenize
import nltk

In [5]:
from keras.preprocessing.text import Tokenizer
from gensim.models.fasttext import FastText
import matplotlib.pyplot as plt
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from nltk import WordPunctTokenizer

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
from nltk.translate.bleu_score import sentence_bleu
def bleu_score(reference,candidate):
  reference=[word_tokenize(reference)] 
  candidate=word_tokenize(candidate)
  score = sentence_bleu(reference, candidate)
  return score

In [7]:
def isint(word):
  try:
    word=int(word) 
  except:
    word=word 
  if type(word)==int and int(word)==float(word):
    return 1 
  else:
    return 0
def isNan(word):
    try:
        word=int(word) 
    except:
        word=0 
    return word
def isfloat(word):
  try:
    word=float(word) 
  except:
    word=word 
  if type(word)==int or type(word)==float:
    return 1 
  else:
    return 0 
def padd(sent):
    sent_arr=word_tokenize(sent)
    for i in range(max(15-len(sent_arr),0)):
        sent_arr.append('<pad>')
    return sent_arr[:15] 
def get_pos(arr):
  ans=[]
  for i in range(len(arr)):
    if arr[i]>0.1:
      ans.append(i) 
  return ans


In [8]:
clean_mp={'6gb': '6 gb', '34gb': '34 gb', '128gb': '128 gb', '3gb': '3 gb','16-megapixel':'16 mp','with15mp': 'with 15 mp', '10mp': '10 mp', '8mp': '8 mp', 'mp-primary': 'mp for primary', 'mp-secondary': 'mp for secondary', '13mp': '13 mp', '16mp': '16 mp', '23mp': '23 mp', '20mp': '20 mp', '12mp': '12 mp','4gb':'4 gb','dualsim':'dual sim','feauture':'feature','gread':'great','..':'.','resposnse':'response','dscount':'discount','ofcourse':'sure','non-smartphone':'non smartphone','freindly':'friendly','perfec':'perfect','rs9999':'rs 9999','asssist':'assist','9000/-':'9000','speciific':'specific','phonebecause':'phone because','188gms':'188 gms','168gms':'168 gms','possesor':'processor','ramand':'ram and','youngeters':'youngsters','multi-':'multi','than32':'than 32','ssure':'sure','5megapixel':'5 mp','addtional':'additional','widely-adopted':'widely adopted','speciifc':'specific','avilable':'available','appliactions':'appliacations','xiomi':'xiaomi','multisim':'multi sim','applehas':'apple has','buawei':'huawei','specificctions':'specifications','non-smartphones':'non smartphones','spcific':'specific','itune':'itunes','brandyou':'brand you','goood':'good','awsome':'awesome','dual-primary':'dual primary','selfie':'photo','multitaking':'multitasking','widely-adopted':'widely adopted','gioned':'gionee','coolp':'coolpad','adbrand':'brand','advanatge':'advantage','ecpecially':'especially','movies/netflix':'movies and netflix','eqipped':'equipped','samsungmobile':'samsung mobile','samusng':'samsung','260euro':'260 euro', '750euro':'750 euro', '250euro':'250 euro', '550euro':'550 euro', '750euro':'750 euro'}
val_mp={'nokia ':'nokia','2015 ':'2015','2012 ':'2012','300 ':'300','200 ':'200','730':'730','110':'110','310':'310','asus ':'asus','2300':'2300','230':'230','210':'210','poco':'poco','single':'single','dual':'dual','multiple':'multiple','toshiba':'toshiba','june':'june','magenta':'magneta','dark gray metallic':'metallic gray',' 30+':'30','12':'12','metallic gray':'metallic gray','platinum':'platinum','glacier silver':'silver','40+':'40','old':'old','10':'10','7+':'7','purple':'purple','indigo':'indigo','5000+':'5000','all':'all','white$black':'white','2000+':'2000','brown':'brown','non shimmer gold':'gold','space gray':'gray','glossy cyan':'cyan','redmi':'redmi','dell':'dell','more':'more','metal':'metal','xperia':'xperia','titanium':'titanium','mics':'mics',' non sym':'and','amethyst gray':'amethyst gray','yu':'yu','karbonn':'karbonn','100':'100','philips':'philips','yellow':'yellow','1500':'1500','golden':'gold','win':'win','10000':'10000','null':'null','nonsym':'and','good': 'good','red':'red','It has':'yes', ' non android':'ios','black': 'black', 'android': 'and', 'y': 'yes', 'white': 'white', 'grey': 'grey', 'less': 'less', 'high': 'high', '15k': '15k', '10k': '10k', 'latest': 'latest', 'non': 'none', 'no': 'no', 'vivo': 'vivo', 'i': 'i', 'gold': 'gold', '15+': '15', 'windows': 'windows', '15k+': '15k', 'nokia': 'nokia', 'e': 'e', '16k+': '16k', '20k': '20k', '2019': '2019', '2k': '2k', '6gb': '6gb', '15': '15', 'jio': 'jio', 'f': 'f', 'blue': 'blue', '4+': '4', 'silver': 'silver', 'big': 'big', 'n':'no','light': 'light', 'htc': 'htc', 'paradise': 'paradise', 'lenovo': 'lenovo', 'yoga tab 3': 'yoga tab 3', 'yes': 'yes', '400': '400', ' light': 'light', '64gb': '64gb', 'd': 'dual', 't': 't-mobile', 'non black': 'non-black', 'small': 'small', 'non and': 'ios', 'samsung': 'samsung', 'samsung ': 'samsung', 'expensive': 'expensive', '128': '128', 'non silver': 'non-silver', '16+': '16', '2016': '2016', '500': '500', '2017': '2017', '128 gb': '128gb', '800+': '800', 'apple': 'apple', '700': '700', '20': '20', 'and': 'and', 'black&blue': 'black', '650': '650', '64+': '64', 'may': 'may', '4000+': '4000', 'pink': 'pink', 'good  ': 'good', '400+': '400', '64': '64', 'non gunmetal': 'non-gunmetal', 'sony': 'sony', 'xperia xa ultra': 'xperia xa ultra', 'non oppo': 'vivo', 'lg': 'lg', 'acer': 'acer', 'iconia tab a700 ': 'iconia tab a700', 'non grey': 'non-grey', 'oppo': 'oppo', '350': '350', 'non crown gold': 'non-gold', 'pearl white\n': 'white', 'recent': 'recent', '300': '300', '20+': '20', '32+': '32', 'blue||gold': 'blue||gold', '8000+': '8000', 'motorola': 'motorola', '150': '150', '12+': '12', '6': '6', '3': '3', '32': '32', 'galaxy note7': 'galaxy note7', '3000+': '3000', 'null': 'null', '13+': '13', '250': '250', 'aquamarine green': 'green', 's': 's', '200': '200', 'pearl white': 'white', 'huawei': 'huawei', 'u8150 ideos': 'u8150 ideos', 'blackberry': 'blackberry', '6000+': '6000', 'good      ': 'good', 'm': 'm', '16': '16', 'onyx black': 'black', 'green': 'green', 'heavy': 'heavy', 'bla': 'bla', ' 300': '300', '220': '220', 'sym': 'sym', 'iconia tab a501\n': 'iconia tab a501', ' non and': 'ios', 'mic': 'mic', 'non dual': 'non-dual', 'titanium grey': 'grey', 'xiaomi': 'xiaomi', '800': '800', 'ios': 'ios', '64 gb': '64gb', '16 ': '16', '': '', 'chrome': 'chrome', 'gray': 'gray', 'non blue': 'non-blue', 'sony ericsson': 'ericsson', 'xperia arc s': 'xperia', '3500+': '3500', 'good     ': 'good', '10+': '10', 'asus': 'asus', '450': '450', '340': '340', 'leeco\n': 'leeco', '160': '160', 'medium': 'medium', 'good        ': 'good', '2014': '2014', 'non gold': 'non-gold', '600': '600', '4': '4', '2015': '2015', '270': '270', '4.65 hd': 'hd', ' 250': ' 250', 'average': 'average', 'good       ': 'good', 'dual': 'dual', ' and': 'and', 'non vga': 'non-vga', 'orange': 'orange', 'xiamo': 'xiaomi', '1800+': '1800', 'gionee': 'gionee', '5': '5', '140': '140', 'non coolpad': 'non-coolpad', 'alcatel': 'alcatel', 'allview': 'allview', 'zte': 'zte', 'xiaomo ': 'xiaomi', '500+': '500', 'moderate': 'moderate', 'coolpad': 'coolpad', 'gigabyte': 'gigabyte', '2+': '2', 'tiz': 'tiz', 'micromax': 'micromax', '13': '13', 'non white': 'non-white', '280': '280', '3000': '3000', 'vodafone': 'vodafone', 'charcoal black\n': 'black', 'oneplus': 'oneplus', 'vertu': 'vertu', '8': '8', '260': '260', '8+': '8', ' bla': ' bla', '3+': '3', 'meizu': 'meizu', 'archos': 'archos', 'google': 'google', '4500+': '4500', 'hp': 'hp'} 
intent_mp={'Phn_SelReq': 'selling', 'Booking_Req': 'booking'}
u_action_mp={'Image_Q':'image','INternal_RAM_Q':'ram','Image_Q':'image','Model_Q':'model','Brand_Q':'brand','Greet':'greet','null':'null','Phn_SelReq': 'phone', 'RecSlt':'rec_slt','Rec_Slt': 'rec_slt', 'Cost_Q': 'cost', 'Color_Q': 'color', 'Feedback_Slt': 'feedback', 'Feedback': 'feedback', 'Booking_Req': 'booking', 'Thanks': 'thanks', 'DualSim_Q': 'dualsim','Sim-Q':'sim','Released-Yr_Q':'year','Internal RAM_Q':'ram','Internal_RAM_Q':'ram','Release-Date_Q':'year','Colour_Q':'color','P_Camaera_Q':'camera','Display_ Size_Q':'display','Released _Year_Q':'year','Released_Yr-Q':'year','Release-Year_Q':'year','Display_Size-Q':'display','battery_Q':'battery','S_Camera,P_Camera_Q':'camera', 'Release_Year_Q':'year','P_Camera_A,S_Camera_A':'camera','Released_Yr_Q': 'year','weight_g_Q':'weight', 'OS_Q': 'os','P_Camera-Q,S_Camera-Q':'camera', 'P_Camera_Q,S_Camera_Q ':'camera','Display_Q':'display','Camera_Q': 'camera', 'Rating_Q': 'rating', 'Confirm_Q': 'confirm', 'Battery_Q': 'battery', 'Memory_Q': 'memory', 'DuslSim_Q': 'dualsim', 'Review_Q': 'review', 'Specification_Q': 'specification', 'Discount_Q': 'discount', 'Weight_g_Q': 'weight', 'Display_Size_Q': 'display', 'Radio_Q': 'radio', 'Internal_RAM_Q,RAM_Q': 'ram','Released _Yr_Q':'year', 'P_Camera_Q': 'camera', 'Internal_RAM__Q':'ram','Internal_RAM_Q': 'ram','Internal_RAM__Q':'ram', 'Greet': 'greet', 'P_Camera_Q,S_Camera_Q': 'camera', 'Confirm': 'confirm', 'Released_Yr_Q,Released_Month_Q': 'month', 'RAM_Q,Internal_RAM_Q': 'ram', 'Phn_Info': 'phone', 'S_Camera_Q': 'camera', 'Sim_Q': 'sim', 'RAM_Q': 'ram', 'GPS_Q': 'gps', 'P_Camera,S_Camera_Q': 'camera', 'RAM,_Q,Internal_RAM_Q': 'ram', 'SIM_Q': 'sim','Released Year_Q': 'year', 'BatterY_Q': 'battery', 'display_Size_Q': 'display', 'RAM,Internal_RAM_Q': 'ram', 'Weight_Q': 'weight', 'Internal_RAM,RAM_Q': 'ram', 'Released_Yr__Q': 'year', 'Display_SizeQ': 'display'}
a_action_mp={'Image_Q':'image','Internal_RAM_Q':'ram','Model_A':'model','Specification_D': 'specification', 'Result': 'result', 'Cost_A': 'cost', 'Color_A ': 'color', 'Rephrase': 'rephrase', 'Success': 'success', 'Welcome': 'welcome', 'Color_A': 'color', 'DualSim_A': 'dualsim', 'Released_Yr_A': 'year', 'OS_A': 'os', 'Camera_A': 'camera', 'Rating_A': 'rating', 'Confirm': 'confirm', 'Battery_A': 'battery', 'Memory': 'memory', 'Review_A': 'review', 'Specification_A': 'specification', 'Discount_A': 'discount', 'Confirm_Q': 'confirm', 'Feedback': 'feedback', 'Feedback_R': 'feedback', 'Weight_g_A': 'weight', 'Display_Size_A': 'size', 'Radio_A': 'radio', 'Internal_RAM_A': 'ram', 'RAM_A': 'ram', 'P_Camera_A': 'camera', 'S_Camera_A': 'camera', 'ReleasedYear_A': 'year', 'ReleasedMonth_A': 'month', 'RAM_Q': 'ram', 'Greet': 'greet', 'P_Camera_Q': 'camera', 'Sim_A': 'sim', 'Cost_Q': 'cost', 'GPS_A': 'gps', 'Released_Month_A': 'month', 'Weight_g_Q': 'weight', 'P_Camera': 'camera', 'null': 'null', 'SIM_A': 'sim','Memory_A':'memory'} 
correct={'Released Year':'year',' Color':'color','Released_Year':'year','color':'color','Display':'display','Weigh':'weight','null':'null','Camera': 'camera', 'Battery': 'battery', 'Color': 'color', 'OS': 'os', 'Dualsim': 'sim', 'Touch': 'touch', 'Memory': 'memory', 'Cost': 'cost', 'Rating': 'rating', 'Price': 'price', 'Released_Yr': 'year','Released _Yr':'year','Released _Year':'year','Released':'year', 'Release':'year','Company': 'company', 'Model': 'model', 'Total_Review': 'review', 'DualSim': 'sim', 'RAM': 'ram', 'Display_Size': 'size', 'Weight_g': 'weight', 'Brand': 'brand', 'Radio': 'radio', 'Internal_RAM': 'ram', 'Sim': 'sim', 'P_Camera': 'camera', 'S_Camera': 'camera', 'Internal_Ram': 'ram', 'GPS': 'gps', 'Internal RAM': 'ram', 'Released_ Yr': 'year', 'Released_Month': 'month', 'SIM': 'sim', 'weight_g': 'weight', 'Weight_G': 'weight', 'Displa_Size': 'size', ' Internal_RAM': 'ram', 'nll': 'nll', 'battery': 'battery', 'Dsplay_Size': 'size', ' Model': 'model', 'Weight': 'weight', 'Apple': 'apple', ' P_Camera': 'camera'}

In [9]:
mem_arr=['256','512','128','64','32','48','16','12','10','4','6','1','2','3','8']
year_arr=['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']
month_arr=['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
rating_arr=['4 . 1','4 . 2','4 . 3','4 . 4','4 . 5','4 . 6','4 . 7','4 . 8','4 . 9','5 . 0','1','2','3','4','5'] 
bat_arr=['3800', '41000', '38000', '3000', '2400', '3300', '24', '3260', '1785', '2870', '2700', '7300', '32', '2900', '4000', '2620', '2850', '2300', '4010', '2950', '2500', '6020', '3505', '2730', '5000', '304', '2770', '4100', '1830', '2915', '2650', '1200', '2000', '4550', '2100', '1750', '400', '1960', '2050', '2800', '2600', '2530', '2750', '1950', '2110', '2910', '3020', '2760', '4800','3400.0', '4080.0', '4020.0', '2000.0', '2870.0', '2000.0', '2000.0', '4000.0', '2420.0', '4550.0', '2300.0', '2000.0', '1300.0', '1300.0', '2000.0', '2300.0', '2000.0', '2700.0', '2100.0', '3500.0', '2500.0', '1300.0', '4600.0', 'nan', 'nan', 'nan', '2955.0', '2955.0', '4000.0', '2000.0', '3300.0', '1500.0', '2400.0', '7300.0', '4960.0', '4960.0', '2000.0', '1300.0', '2000.0', '1760.0', '2640.0', '2710.0', '3420.0', '1300.0', '1500.0', '1500.0', '1300.0', '3260.0', '3260.0', '9800.0', '9800.0', '9800.0', '9800.0', 'nan', '3260.0', '1500.0', '3260.0', '1530.0', '1530.0', '1300.0', '1300.0', '1300.0', '1500.0', '1500.0', '1500.0', '1500.0', '1400.0', '1350.0', '1090.0', '1090.0', '970.0', '1500.0', '1350.0', '1530.0', '1530.0', '1530.0', '2460.0', '2500.0', '2150.0', '2000.0', '3000.0', '3240.0', '3000.0', '3000.0', '2610.0', '2580.0', '910.0', '2000.0', '2500.0', '3000.0', '3500.0', '2000.0', '2910.0', '2000.0', '3240.0', '2820.0', '2820.0', '4600.0', '1400.0', '1400.0', '1300.0', '1400.0', '1300.0', '2000.0', '2000.0', '2000.0', 'nan', '1400.0', '4100.0', '4060.0', '3100.0', '1300.0', '1800.0', '1300.0', '3240.0', '3240.0', '3400.0', '3000.0', '2000.0', '1700.0', '1300.0', '2150.0', '2000.0', '1700.0', '1000.0', '2500.0', '2500.0', '3240.0', '1800.0', '1400.0', '1900.0', '1800.0', '1300.0', '1300.0', '2000.0', '3400.0', '2000.0', '1700.0', '2000.0', '1400.0', '2500.0', 'nan', 'nan', 'nan', '3300.0', '1800.0', '1800.0', '1400.0', '1400.0', '1800.0', '2500.0', '2500.0', '1300.0', '1800.0', '1800.0', '1400.0', '1300.0', '1500.0', '1300.0', '1300.0', '3000.0', '3000.0', '5020.0', '3150.0', '2350.0', '2350.0', '2400.0', '3150.0', '3150.0', '2000.0', '2000.0', '4000.0', '1700.0', '3140.0', '6000.0', '4000.0', '2100.0', '1850.0', '3500.0', '1400.0', '1800.0', '1350.0', '1920.0', '2400.0', '2200.0', '2050.0', '1420.0', '2000.0', '1750.0', '1500.0', '2300.0', '2200.0', '2500.0', '2500.0', '4000.0', '2100.0', '3000.0', '2000.0', '1700.0', '1800.0', '1800.0', '1800.0', '3000.0', '1500.0', '2100.0', '2100.0', '1600.0', '1700.0', '1600.0', '1900.0', '1400.0', 'nan', '2400.0', 'nan', 'nan', 'nan', '4100.0', '304.0', '8827.0', '2900.0', '1960.0', '279.0', '1624.0', '2750.0', '1715.0', '10307.0', '5124.0', '7340.0', '6470.0', '2915.0', '1810.0', '8600.0', '6470.0', '1560.0', '1510.0', '4490.0', '11560.0', '1440.0', '11560.0', '1432.0', '6930.0', '6930.0', '1420.0', '1420.0', 'nan', 'nan', '2400.0', '2950.0', '2100.0', '2000.0', '2700.0', '2800.0', '2000.0', '1400.0', '2650.0', '5000.0', '7800.0', '3300.0', '5000.0', '3480.0', '4680.0', '2600.0', '4100.0', '3000.0', '7800.0', '5900.0', '4130.0', '3000.0', '2600.0', '4100.0', '4600.0', '3000.0', '3000.0', '5000.0', '2070.0', '2070.0', '3010.0', '2500.0', '1600.0', '2070.0', '3000.0', '3000.0', '1800.0', '2070.0', '5000.0', '3000.0', '3000.0', '3000.0', '3000.0', 'nan', '2400.0', '1300.0', '2500.0', '3030.0', '4000.0', '1300.0', '3000.0', '2500.0', '3000.0', '3000.0', '1500.0', '2100.0', '1500.0', '3000.0', '2500.0', '1900.0', '2060.0', '1500.0', '1500.0', '2110.0', '1750.0', '152.0', '152.0', '1500.0', '152.0', '1500.0', '1500.0', '2500.0', '1900.0', '1500.0', '2300.0', '2300.0', '2400.0', '3230.0', '3300.0', '2110.0', '2110.0', '600.0', '2060.0', '1170.0', '1500.0', '2400.0', '5070.0', '3950.0', '3200.0', '3100.0', '1500.0', '1500.0', '4270.0', '2400.0', '1900.0', '4325.0', '2140.0', '4325.0', '2500.0', '2500.0', '2200.0', '2500.0', '4400.0', '2500.0', '2500.0', '1520.0', '1100.0', '244.0', '1300.0', '1100.0', '1530.0', '1300.0', 'nan', '1840.0', '2520.0', '2100.0', '2000.0', '3000.0', '3505.0', '3000.0', '2610.0', '3410.0', '2800.0', '2100.0', '3450.0', '2515.0', '2500.0', '1800.0', '2880.0', '1450.0', '2180.0', '1800.0', '2100.0', 'nan', '1450.0', '100.0', '1230.0', '1230.0', '1000.0', '1000.0', '1000.0', '1270.0', '1230.0', '1230.0', '1230.0', '1230.0', '5300.0', '1500.0', '1150.0', '1150.0', '1300.0', '1500.0', '1400.0', '1400.0', '1500.0', '900.0', '3000.0', '2000.0', '3500.0', '3000.0', '5000.0', '2000.0', '2100.0', '2100.0', '3200.0', '2200.0', '3000.0', '1400.0', '2200.0', '2000.0', '1800.0', '2300.0', '2000.0', '2500.0', '3000.0', '1800.0', '1300.0', '2000.0', '2100.0', '2350.0', '3000.0', '2000.0', '4500.0', '1800.0', '2000.0', '2000.0', '1800.0', '2020.0', '2000.0', '1600.0', '2500.0', '2250.0', '2000.0', '1500.0', '1000.0', '2000.0', '1600.0', '2800.0', '2000.0', '2000.0', '1500.0', '2600.0', '2000.0', '1800.0', '2200.0', '1760.0', '2800.0', '1800.0', '1800.0', '1500.0', '1280.0', '1600.0', '6000.0', '3500.0', '3000.0', '1300.0', '1600.0', '1110.0', '1600.0', '2500.0', '4250.0', '1200.0', '3100.0', '3100.0', '3080.0', '3080.0', '3050.0', '3200.0', '1800.0', '1460.0', '3000.0', '3000.0', '2000.0', '2630.0', '1400.0', '2000.0', '2100.0', '2500.0', '4060.0', '2500.0', '4070.0', '3050.0', '4010.0', '2500.0', '2500.0', '2500.0', '3000.0', '2800.0', '2000.0', '1800.0', '2500.0', '2500.0', '1800.0', '7000.0', '2100.0', '4100.0', '4100.0', '1400.0', '1250.0', '1100.0', '1400.0', '1530.0', 'nan', '1530.0', 'nan', 'nan', '1530.0', '1530.0', '1530.0', '1530.0', '1530.0', '1530.0', '1530.0', '1100.0', '1300.0', '2100.0', '1600.0', '2000.0', '2000.0', '1400.0', '2000.0', '1700.0', '2000.0', '1900.0', '1700.0', '1500.0', '1500.0', '1550.0', '1500.0', '1260.0', '1260.0', '1300.0', '1300.0', '920.0', '3450.0', '6020.0', '4550.0', '4010.0', '4000.0', '7000.0', '4000.0', '3000.0', '3100.0', '3150.0', '3130.0', '1800.0', '2530.0', '3000.0', '5000.0', '5020.0', '4000.0', '2000.0', '3150.0', '2400.0', '3500.0', '5000.0', '6020.0', '2700.0', '5000.0', '2050.0', '1950.0', '1800.0', '1800.0', '2300.0', '3450.0', '2770.0', '4150.0', '2100.0', '285.0', '2100.0', '3000.0', 'nan', '4100.0', '3000.0', '5680.0', '7000.0', '4100.0', '1230.0', 'nan', '1260.0', '1140.0', '1840.0', '1590.0', '1200.0', '3000.0', '4000.0', '3000.0', '2500.0', '3200.0', '2200.0', '3000.0', '2700.0', '2300.0', '2800.0', '2200.0', '2840.0', '2800.0', '2840.0', '3000.0', '3000.0', '2700.0', '2200.0', '2200.0', '2000.0', '3000.0', '2840.0', '2800.0', '2600.0', '2700.0', '2150.0', '2840.0', '2000.0', '2000.0', '2840.0', '2600.0', '2000.0', '2840.0', '2840.0', '2800.0', '2800.0', '2840.0', '2600.0', '2000.0', '2000.0', '2000.0', '2600.0', '2100.0', '2100.0', '2100.0', '6700.0', '2600.0', '2600.0', '2400.0', '2040.0', '2600.0', '2600.0', '2600.0', '2600.0', '2100.0', '2600.0', '2600.0', '2700.0', '2100.0', '2600.0', '1950.0', '2600.0', '2110.0', '2000.0', '1300.0', '2600.0', '2600.0', '2000.0', '2000.0', '2600.0', '2600.0', '2040.0', '2100.0', '2100.0', '2100.0', '2100.0', '2100.0', '3300.0', '1650.0', '2100.0', '1800.0', '1800.0', '1800.0', '3200.0', '1230.0', '1860.0', '2000.0', '2300.0', '2300.0', '1800.0', '2020.0', '2020.0', '1800.0', '1800.0', '1620.0', '1800.0', '2100.0', '1800.0', '1800.0', '1700.0', '1800.0', '1800.0', '1650.0', '1650.0', '1650.0', '1650.0', '1650.0', '1230.0', '1810.0', '1700.0', '2000.0', '2000.0', '1800.0', '1800.0', '1800.0', '1650.0', '1650.0', '1620.0', '1500.0', '1620.0', '1620.0', '1620.0', '1520.0', '1600.0', '1230.0', '1730.0', '1620.0', '1600.0', '1520.0', '1600.0', '1730.0', '7300.0', '1600.0', '1520.0', '1400.0', '1250.0', '1300.0', '1450.0', '1520.0', '1730.0', '1520.0', '1730.0', '1230.0', '1400.0', '1450.0', '1450.0', '1230.0', '1520.0', '1250.0', '4000.0', '4000.0', '4000.0', '1230.0', '1400.0', '1500.0', '1200.0', '1230.0', '1230.0', '1230.0', '1300.0', '1300.0', '1500.0', '1230.0', '1300.0', '1200.0', '1300.0', '1300.0', '1400.0', '1200.0', '1300.0', '1400.0', '1230.0', '1500.0', '1300.0', '1100.0', '1100.0', '1000.0', '1500.0', '1350.0', '1500.0', '1500.0', '1340.0', '1500.0', '1500.0', '1500.0', '1500.0', '1100.0', '1150.0', '1100.0', '1500.0', '1350.0', '1350.0', '1100.0', '1050.0', '1340.0', '900.0', '2100.0', '1100.0', '1350.0', '1050.0', '1350.0', '2200.0', 'nan', '1500.0', '1250.0', '3200.0', '4000.0', '4800.0', '6660.0', '3340.0', '2950.0', '4800.0', '3100.0', '4000.0', '3020.0', '3000.0', '3000.0', '4000.0', '3750.0', '3200.0', '3000.0', '3000.0', '2900.0', '3020.0', '4000.0', '4000.0', '4000.0', '3500.0', '4100.0', '4800.0', '3340.0', '3100.0', '3340.0', '3020.0', '6660.0', '4360.0', '4100.0', '3340.0', '4500.0', '3000.0', '2200.0', '4100.0', '3000.0', '2100.0', '2200.0', '3000.0', '3400.0', '3000.0', '4000.0', '6660.0', '2200.0', '4000.0', '3000.0', '3000.0', '4000.0', '3450.0', '2700.0', '3100.0', '2200.0', '3100.0', '4800.0', '2000.0', '2000.0', '2550.0', '2200.0', '2200.0', '4360.0', '2680.0', '2200.0', '5000.0', '2000.0', '1730.0', '1750.0', '3500.0', '3600.0', '4100.0', '3000.0', '2000.0', '4800.0', '4800.0', '2000.0', '2000.0', '2000.0', '3000.0', '4100.0', '2000.0', '3100.0', '3000.0', '2300.0', '2000.0', '2000.0', '1500.0', '2500.0', '2000.0', '2300.0', '2100.0', '6600.0', '2000.0', '2000.0', '4800.0', '5000.0', '1750.0', '4100.0', '2000.0', '4050.0', '1350.0', '3000.0', '2300.0', '2400.0', '1730.0', '1700.0', '2150.0', '2150.0', '4100.0', '2000.0', '1730.0', '1650.0', '1700.0', '2150.0', '1950.0', '1730.0', '1500.0', '1950.0', '4050.0', '3000.0', '1700.0', '1930.0', '1400.0', '2230.0', '1400.0', '1400.0', '2000.0', '1400.0', '1930.0', '1500.0', '4100.0', '1050.0', '400.0', '1550.0', '500.0', '1670.0', '1880.0', '1400.0', '6600.0', '1800.0', '2600.0', '1800.0', '1800.0', '1900.0', '4100.0', '4100.0', '1500.0', '1930.0', '1400.0', '1400.0', '1500.0', '1400.0', '1200.0', '3250.0', '1400.0', '1500.0', '1200.0', '1200.0', '1150.0', '2200.0', '1200.0', '1620.0', '1530.0', '1660.0', '600.0', '2500.0', '2100.0', '1900.0', '1700.0', '1400.0', '2300.0', '1800.0', '1800.0', '2000.0', '3100.0', '3100.0', '2500.0', '1370.0', '1500.0', '2000.0', '3020.0', '2620.0', '3000.0', '2500.0', '2900.0', '2500.0', '2560.0', '2500.0', '4000.0', '2100.0', '2000.0', '2100.0', '1800.0', '2600.0', '2000.0', '2000.0', '1400.0', '1400.0', '2000.0', '2000.0', '2000.0', '1500.0', '2000.0', '4000.0', '4070.0', '3000.0', '4070.0', '3100.0', '3000.0', '3000.0', '3400.0', '3000.0', '7000.0', '7000.0', '4850.0', '4850.0', '4250.0', '3100.0', '2300.0', '2300.0', '2000.0', '2000.0', '5100.0', '4000.0', '4000.0', '3000.0', '9300.0', '1700.0', '3500.0', '2750.0', '4050.0', '4050.0', '4050.0', '3500.0', '2300.0', '3100.0', '7000.0', '4290.0', '3450.0', '2750.0', '2750.0', '2900.0', '4900.0', '3500.0', '2750.0', '2700.0', '3300.0', '3500.0', '2300.0', '10200.0', '2000.0', '4250.0', '3500.0', '5000.0', '2300.0', '2300.0', '4000.0', '2420.0', '4100.0', '2000.0', '2300.0', '4000.0', '2150.0', '2300.0', '1500.0', '3000.0', '3000.0', '2900.0', '2900.0', '4000.0', '4000.0', '3450.0', '3450.0', '3450.0', '2300.0', '4000.0', '2410.0', '2300.0', 'nan', '2500.0', '2500.0', '1500.0', '2300.0', '2150.0', '2000.0', '9600.0', '9600.0', '6400.0', '4290.0', '2300.0', '3000.0', '2500.0', '4000.0', '2500.0', '2000.0', '3000.0', '2000.0', '2500.0', '2000.0', '1300.0', '2000.0', '2000.0', '6340.0', '4200.0', '3450.0', '3500.0', '9000.0', '4000.0', '2000.0', '3000.0', '2500.0', '2250.0', '3000.0', '2000.0', '3000.0', '9000.0', '6000.0', '2500.0', '2000.0', '2000.0', '2250.0', '4000.0', '2000.0', '1500.0', '2250.0', '6350.0', '3500.0', '3500.0', '2500.0', '2250.0', '3550.0', '2000.0', '2000.0', '1500.0', '2000.0', '3500.0', '2250.0', '1500.0', '2250.0', '1500.0', '1900.0', 'nan', '7400.0', '4100.0', '3300.0', '3080.0', '3200.0', '2800.0', '2700.0', '2800.0', '2500.0', '2500.0', '2100.0', '6000.0', '3200.0', '2100.0', '2800.0', '3000.0', 'nan', '4100.0', '2100.0', '3000.0', '1900.0', '1940.0', '2800.0', '2800.0', '2520.0', '2300.0', '2125.0', '3000.0', '2300.0', '2125.0', '1940.0', '3000.0', '3000.0', '3000.0', '2700.0', '2050.0', '7400.0', '4200.0', '1700.0', '1900.0', '2540.0', '2300.0', '2540.0', '3000.0', '3000.0', '3000.0', '3000.0', '2610.0', '2540.0', '2100.0', '1900.0', '1900.0', '3000.0', '2100.0', '1820.0', '3000.0', '3000.0', '2100.0', '1540.0', '3000.0', '2540.0', '1900.0', '4200.0', '3200.0', '2610.0', '4000.0', '1900.0', '3200.0', '3000.0', '2540.0', '2100.0', '3000.0', '3000.0', '3000.0', '8000.0', '8000.0', '4200.0', '4000.0', '2540.0', '2440.0', '2100.0', '4600.0', '2440.0', '2440.0', '2440.0', '2440.0', '2540.0', '2540.0', '2100.0', '2100.0', '3200.0', '1540.0', '2460.0', '3140.0', '2300.0', '3500.0', '3140.0', '3140.0', '1540.0', '4600.0', '3000.0', '2150.0', '2460.0', '2280.0', '1700.0', '1700.0', '1540.0', '2460.0', '2460.0', '2540.0', '2460.0', '2150.0', '3140.0', '2460.0', '2460.0', '1700.0', '1700.0', '1540.0', '1540.0', '1540.0', '2100.0', '2150.0', '1700.0', '2150.0', '2100.0', '2100.0', '2100.0', '2080.0', '1700.0', '2150.0', '2150.0', '1700.0', '2100.0', '1540.0', '1520.0', '2150.0', '1830.0', '1700.0', '1300.0', '2150.0', '1520.0', '1520.0', '1700.0', '1500.0', '2080.0', '1500.0', '1540.0', '1700.0', '1830.0', '1540.0', '1830.0', '1830.0', '1500.0', '1830.0', '1830.0', '1500.0', '1500.0', '1500.0', '1500.0', '1500.0', '1500.0', '1500.0', '1500.0', '1500.0', '1500.0', '1500.0', '1500.0', '1500.0', '1500.0', '1500.0', '1500.0', '1500.0', '1500.0', '1500.0', 'nan', '1500.0', '1500.0', '1500.0', '3000.0', '3400.0', '3000.0', '4000.0', '3400.0', '3150.0', '2760.0', '3260.0', '3060.0', '3070.0', '4100.0', '3100.0', '3060.0', '3020.0', '2870.0', '2560.0', '4100.0', '3140.0', '3050.0', '2500.0', '3150.0', '3100.0', '3140.0', '2500.0', '3350.0', '3100.0', '2400.0', '1800.0', '1700.0', '1600.0', '4000.0', '1800.0', '2000.0', '2100.0', '2000.0', '2000.0', '4400.0', '3200.0', '1950.0', '2450.0', '2000.0', '2600.0', '2000.0', '2000.0', '2100.0', '2000.0', '2100.0', '2000.0', '1400.0', '2000.0', '1600.0', '1400.0', '1450.0', '1500.0', '2000.0', '3340.0', '3340.0', '3000.0', '3000.0', '2200.0', '3000.0', '3000.0', '3000.0', '3000.0', '2500.0', '2500.0', '2500.0', '1905.0', '1905.0', '1050.0', '5000.0', '5000.0', '2800.0', '2800.0', '3000.0', '4000.0', '2350.0', '3000.0', '2800.0', '3050.0', '3500.0', '3510.0', '2800.0', '3500.0', '2600.0', '3000.0', '3000.0', '2800.0', '2470.0', '3760.0', '3760.0', '3630.0', '3000.0', '3630.0', '3630.0', '2470.0', '2470.0', '2390.0', '2390.0', '2390.0', '2390.0', '3900.0', '3900.0', '3220.0', '2300.0', '2070.0', '2070.0', '2070.0', '2000.0', '1980.0', '1980.0', '2070.0', '2070.0', '2200.0', '2130.0', '3500.0', '2000.0', '2000.0', '1785.0', '2000.0', '2000.0', '3300.0', '2530.0', '530.0', '2000.0', '2000.0', '1650.0', '1780.0', '1785.0', '1780.0', '1650.0', 'nan', '1700.0', '1750.0', '1780.0', '1540.0', 'nan', '3300.0', '1650.0', '1785.0', '3300.0', '1390.0', 'nan', '1400.0', '1650.0', '1390.0', '3960.0', '7000.0', '1500.0', '1390.0', '3960.0', '3960.0', '7000.0', '7000.0', '248.0', '1780.0', '1780.0', '1785.0', '1860.0', 'nan', '1600.0', '1700.0', '1540.0', '1420.0', '1420.0', '1420.0', '1540.0', '1540.0', '1400.0', '1700.0', '1880.0', '1390.0', '1500.0', '1880.0', '1420.0', 'nan', 'nan', '2500.0', '1930.0', '1930.0', '1420.0', '1735.0', '1540.0', '1930.0', '1500.0', '1390.0', '1420.0', '1420.0', '1170.0', '1170.0', '1170.0', '1380.0', '1540.0', '1150.0', '1540.0', '1400.0', '1130.0', '1540.0', '1540.0', '1540.0', '1390.0', '1390.0', '1270.0', '1270.0', '1130.0', '1400.0', '1390.0', '1140.0', '1390.0', '1400.0', '1420.0', '2400.0', '1170.0', '1130.0', '1170.0', '1170.0', '1900.0', '1200.0', '1650.0', '1000.0', '1000.0', '3000.0', '1830.0', '2200.0', '2200.0', '2200.0', '1430.0', '1430.0', '1800.0', '2420.0', '1830.0', '1830.0', '1830.0', '2000.0', '1500.0', '1500.0', '2420.0', '1430.0', '3400.0', '3400.0', '8120.0', '2000.0', '2000.0', '2000.0', '2000.0', '2000.0', '1430.0', '1300.0', '1300.0', '1800.0', '1300.0', '1800.0', '2000.0', '1650.0', '1300.0', '1400.0', '1450.0', '1300.0', '1830.0', '1830.0', '1300.0', '1450.0', '1300.0', '1300.0', '300.0', '1000.0', '1000.0', '1000.0', '1300.0', '1080.0', '1110.0', '1450.0', '1050.0', '1200.0', '1200.0', '1200.0', '1500.0', '1200.0', '1000.0', '1200.0', '1050.0', '1200.0', '1320.0', '1200.0', '1200.0', '1050.0', '1320.0', '1320.0', '1200.0', '1200.0', '1320.0', '1320.0', '1320.0', '1500.0', '1000.0', '1200.0', '1500.0', '1320.0', '1500.0', '1000.0', '950.0', '1200.0', '900.0', '950.0', '1050.0', '1200.0', '900.0', '950.0', '5200.0', '1975.0', '1975.0', '3300.0', '3400.0', '3000.0', '2525.0', '3300.0', '3100.0', '4000.0', '3000.0', '3200.0', '3200.0', '4000.0', '2900.0', '3075.0', '4000.0', '3010.0', '2630.0', '3075.0', '4120.0', '2850.0', '2500.0', '2400.0', '2420.0', '3070.0', '2320.0', '2000.0', '2420.0', '2000.0', '4100.0', '2320.0', '2000.0', '2000.0', '1700.0', '2000.0', '2000.0', '2420.0', '2000.0', '3000.0', '2410.0', '1900.0', '2000.0', '1700.0', '1900.0', '2140.0', '2410.0', '3000.0', '2800.0', '1900.0', '2410.0', '3610.0', '2000.0', '2500.0', '2020.0', '1520.0', '3150.0', '1710.0', '1700.0', '1500.0', '1710.0', 'nan', 'nan', 'nan', '1150.0', '1150.0', '1500.0', '4000.0', '4000.0', '4000.0', '3000.0', '4000.0', '3000.0', '3000.0', '2200.0', '2600.0', '2500.0', '2000.0', '2800.0', '2450.0', '3000.0', '4000.0', '3200.0', '2350.0', '2500.0', '2910.0', '3500.0', '2000.0', '1980.0', '2500.0', '2100.0', '2800.0', '1500.0', '2500.0', '2000.0', '2600.0', '2000.0', '2500.0', '1300.0', '1700.0', '2000.0', '1500.0', '2500.0', '2000.0', '1800.0', '4590.0', '2100.0', '2600.0', '1830.0', '2020.0', '1830.0', '1650.0', '6400.0', '1320.0', '1400.0', '3300.0', '2400.0', '4600.0', '2600.0', '2300.0', '2000.0', '2000.0', '1500.0', '1500.0', '3500.0', '2000.0', '1700.0', '2120.0', '1950.0', '5000.0', '2400.0', '2000.0', '2050.0', '2000.0', '2500.0', '2300.0', '2000.0', '2600.0', '2000.0', '3300.0', '2400.0', '3000.0', '3500.0', '2800.0', '6000.0', '1500.0', '2600.0', '3300.0', '3600.0', '3000.0', '2350.0', '2600.0', '2600.0', '4000.0', '3300.0', '3300.0', '3300.0', '2400.0', '3300.0', '1500.0', '3500.0', '3500.0', '3000.0', '2600.0', '4000.0', '2600.0', '2600.0', '4000.0', '2600.0', '3300.0', '2600.0', '5000.0', '3300.0', '3100.0', '7300.0', '4000.0', '3000.0', '3600.0', '3600.0', '3000.0', '1500.0', '5000.0', '2050.0', '4000.0', '3300.0', '2900.0', '2300.0', '2600.0', '5700.0', '3000.0', '2600.0', '2600.0', '1900.0', '3000.0', '3000.0', '3000.0', '3000.0', '3000.0', '3000.0', '2800.0', '1900.0', '1800.0', '5870.0', '4000.0', '3050.0', '3050.0', '1500.0', '3000.0', '2600.0', '6800.0', '5000.0', '3500.0', '3600.0', '6000.0', '4200.0', '2200.0', '2600.0', '2550.0', '2600.0', '2550.0', '2550.0', '1850.0', '1850.0', '2000.0', '3600.0', '1500.0', '2600.0', '2600.0', '2500.0', '2950.0', '2000.0', '2300.0', '2300.0', '1900.0', '1900.0', '2800.0', '1200.0', '1500.0', '2600.0', '2600.0', '1900.0', '3000.0', '3000.0', '3220.0', '3220.0', '4450.0', '4450.0', '2800.0', '2800.0', '1860.0', '1860.0', '2100.0', '2100.0', '1500.0', '1500.0', '2100.0', '1800.0', '1500.0', '1300.0', '2000.0', '2800.0', '2800.0', '4900.0', '4900.0', '7900.0', '7900.0', '2000.0', '2100.0', '3200.0', '2800.0', '2430.0', '2100.0', '1500.0', '4000.0', '4000.0', '4000.0', '4450.0', '4450.0', '4450.0', '6800.0', '6800.0', '6800.0', '2100.0', '1500.0', '2800.0', '2800.0', '2800.0', '2800.0', '2100.0', '2100.0', '1300.0', '3100.0', '3100.0', '3600.0', '3600.0', '2100.0', '9500.0', '9500.0', '9500.0', '9500.0', '9500.0', '9500.0', '8220.0', '8220.0', '4800.0', '4800.0', '2000.0', '2000.0', '2600.0', '2600.0', '2100.0', '1500.0', '2600.0', '1820.0', '2100.0', '2600.0', '1800.0', '2800.0', '1500.0', '8220.0', '3200.0', '1750.0', '2000.0', '2330.0', '1500.0', '1900.0', '2600.0', '4450.0', '6800.0', '6800.0', '6800.0', '1500.0', '1800.0', '4000.0', '4000.0', '3200.0', '2600.0', '2000.0', '1200.0', '2600.0', '2600.0', '2600.0', '2600.0', '4600.0', '4600.0', '1300.0', '1300.0', '2000.0', '1700.0', '1650.0', '2100.0', '2100.0', '2100.0', '7000.0', '2100.0', '1800.0', '1300.0', '2100.0', '2100.0', '9000.0', '8200.0', '1500.0', '1300.0', '1300.0', '1850.0', '2000.0', '2100.0', '2100.0', '1800.0', '3100.0', '3100.0', '2300.0', '1650.0', '1750.0', '2100.0', '1700.0', '4000.0', '7000.0', '7000.0', '1500.0', '1650.0', '1300.0', '1300.0', '1300.0', '6100.0', '1500.0', '2100.0', '2100.0', '2100.0', '2100.0', '2000.0', '7000.0', '7000.0', '1650.0', '1500.0', '1500.0', '1300.0', '1200.0', '4000.0', '4000.0', '1500.0', '1750.0', '1750.0', '5100.0', '1850.0', '1750.0', '2500.0', '2500.0', '1300.0', '1800.0', '1500.0', '1650.0', '1500.0', '1850.0', '1500.0', '1850.0', '1750.0', '1650.0', '1800.0', '1650.0', '1200.0', '4000.0', '1500.0', '1850.0', '5100.0', '2500.0', '1650.0', 'nan', '1850.0', '1800.0', '1850.0', '1850.0', '6100.0', '1500.0', '1200.0', '1500.0', '1700.0', '1650.0', '1500.0', '1500.0', '1500.0', '1650.0', '1160.0', '1500.0', '1500.0', '7000.0', '6800.0', '7000.0', '6000.0', '6000.0', '1500.0', '1650.0', '1650.0', '1350.0', '1350.0', '1350.0', '1200.0', '1650.0', '1500.0', '4000.0', '4000.0', '1500.0', '1500.0', '1500.0', '1500.0', '1500.0', '1500.0', '1500.0', '1500.0', '1800.0', '1500.0', '1500.0', '1440.0', '1480.0', '1500.0', '1500.0', '1480.0', '1480.0', '1500.0', '1000.0', '1500.0', '1800.0', '1200.0', '1500.0', '1440.0', '1300.0', '1480.0', '1700.0', '3000.0', '2600.0', '3000.0', '2040.0', '1950.0', '1240.0', '4800.0', '4800.0', '2620.0', '2900.0', '3230.0', '2700.0', '2300.0', '2700.0', '2900.0', '2300.0', '2700.0', '2700.0', '2620.0', '2300.0', '2300.0', '3430.0', '3430.0', '2900.0', '2900.0', '2700.0', '2600.0', '2600.0', '2930.0', '2930.0', '2930.0', '2930.0', '2600.0', '2600.0', '2400.0', '2400.0', '6000.0', '6000.0', '2300.0', '2300.0', '2300.0', '2300.0', '2330.0', '2330.0', '4500.0', '3100.0', '3200.0', '3100.0', '2600.0', '2300.0', '2500.0', '2500.0', '3000.0', '2500.0', '2300.0', '2300.0', '6000.0', '6000.0', '3200.0', '1750.0', '1700.0', '3000.0', '3000.0', '3000.0', '2300.0', '3000.0', '3050.0', '2390.0', '1750.0', '2300.0', '1750.0', '2370.0', '6000.0', '6000.0', '2330.0', '2370.0', '1530.0', '1530.0', '1850.0', '6000.0', '6000.0', '1750.0', '1750.0', '1750.0', '1850.0', '1750.0', '1500.0', '1500.0', '1500.0', '1305.0', '1910.0', '1500.0', '1700.0', '1910.0', '1900.0', '1500.0', '1320.0', '1320.0', '1305.0', '1750.0', '1900.0', '1900.0', '3080.0', '3080.0', 'nan', 'nan', '1500.0', '1500.0', '1200.0', '1500.0', '1200.0', '1200.0', '1200.0', '1200.0', '1500.0', '1500.0', '1500.0', '1500.0', '1500.0', '1200.0', '1500.0', '1500.0', '1500.0', '1000.0', '1500.0', '1700.0', '2100.0', '2000.0', '2550.0', '2550.0', '2400.0', '1420.0', '4200.0', '1400.0', '1750.0', '1500.0', '1500.0', '1500.0', '2000.0', '1520.0', '1500.0', '1500.0', '1500.0', '1500.0', '1400.0', '1300.0', '1150.0', '1340.0', '1230.0', '1350.0', '1500.0', '1340.0', '1150.0', '3300.0', '3300.0', '2500.0', '2500.0', '3800.0', '2500.0', '1500.0', 'nan', '1500.0', 'nan', '1000.0', '1530.0', '1320.0', '1050.0', '3160.0', '2275.0', '2275.0', '2200.0', '2500.0', '2600.0', '1850.0', '2600.0', '1800.0', '1700.0', '1650.0', '1900.0', '2100.0', '1400.0', '1450.0', '2000.0', '2500.0', '3000.0', '2730.0', '3160.0', '3000.0', '4080.0', '4000.0', '3050.0', '3000.0', '4000.0', '3000.0', '3000.0', '2550.0', '3000.0', '2400.0', '3600.0', '3600.0', '2350.0', '2100.0', '4150.0', '2300.0', '2100.0', '2400.0', '5830.0', '2000.0', '3000.0', '2960.0', '2540.0', '1780.0', '3000.0', '4600.0', 'nan', '1400.0', '1880.0', '3000.0', '1880.0', '1400.0', 'nan', '6340.0', '3550.0', '1200.0', '1200.0', '1800.0', '2000.0', '1300.0', '2500.0', '2350.0', '2000.0', '2000.0', '2350.0', '5300.0', '4100.0', '3350.0', '6600.0', '2860.0', '4100.0', '4100.0', '3120.0', '4100.0', '4100.0', '4400.0', '4070.0', '3800.0', '3200.0', '4100.0', '4050.0', '4100.0', '4100.0', '4100.0', '4100.0', '3000.0', '4850.0', '3260.0', '4050.0', '4100.0', '3100.0', '4000.0', '3080.0', '3060.0', '2200.0', '3120.0', '3000.0', '3000.0', '2200.0', '3080.0', '3100.0', '3080.0', '3100.0', '3050.0', '2000.0', '2000.0', '2000.0', '2000.0', '1930.0', '1800.0', '1400.0', '1400.0', '1810.0', '4100.0', '1450.0', '2100.0', '1400.0', '2600.0', '1800.0', '2400.0', '4000.0', '4000.0', '2500.0', '1400.0', '2000.0', '2000.0', '1900.0', '2000.0', '1500.0', '2100.0', '2100.0', '2400.0', '2000.0', '2500.0', '1800.0', '3000.0', '2000.0', '4000.0', '3000.0', '2000.0', '2230.0', '2500.0', '3200.0', '3400.0', '3990.0', '2950.0', '2500.0', '5000.0', '3140.0', '2730.0', '4100.0', '3000.0', '2705.0', '3000.0', '5000.0', '4000.0', '3250.0', '3000.0', '1400.0', '2800.0', '2150.0', '3000.0', '2540.0', '2500.0', '2200.0', '3080.0', '2300.0', '3000.0', '3000.0', '2500.0', '2800.0', '3000.0', '2200.0', '3000.0', '3400.0', '3000.0', '1800.0', '2480.0', '2000.0', '2100.0', '2200.0', '2900.0', '3000.0', '2900.0', '2900.0', '1850.0', '1200.0', '2400.0', '2400.0', '2400.0', '2000.0', '2500.0', '3400.0', '1850.0', '2000.0', '3100.0', '2300.0', '2000.0', '1400.0', '2000.0', '2300.0', '4250.0', '1400.0', '3100.0', '2300.0', '2000.0', '2300.0', '2070.0', '2000.0', '1650.0', '2500.0', '2500.0', '2000.0', '2500.0', '3200.0', '1500.0', '1780.0', '1730.0', '1500.0', '2300.0', '1780.0', '1800.0', '1600.0', '1650.0', '1900.0', '1600.0', '1600.0', '1400.0', '1650.0', '1200.0', '4000.0', '3400.0', '1400.0', '1800.0', '3400.0', '1100.0', '1250.0', '1100.0']
cam_arr=['13', '15', '10', '20', '16', '12', '23', '7', '5', '19', '8', '21', '4', '22', '64', '2', '1']
os_arr=['android','and','ios','mic','windows','blackberry','symphony','mm','coloros','fir']
color_arr=['black', 'mystic black', 'pure white', 'various', 'aquamarine green', 'graphite black', 'titan black', 'titanium black', 'silver', 'white', 'essential white', 'rock black', 'gentle black', 'titanium gray', 'blue', 'soft-touch black', 'satin gold', 'venus gold', 'halo gold', 'gold', 'volcanic grey', 'slate', 'yellow', 'midnight blue', 'lemon yellow', 'dark gray', 'full white', 'bluish black', 'mozilla orange', 'cloudy', 'red', 'green', 'cranberry pink', 'blue', 'flash red', 'twilight blue', 'reddish black', 'ardesia', 'dark blue', 'space gray', 'jet black', 'black(8', 'shimmer gold', 'navy black', 'glacier grey', 'glacier gray', 'gray', 'titanium grey', 'sand gold', 'sapphire black', 'glacier silver', 'charcoal black', 'rosa', 'pure black', 'rose gold', 'osmium black', 'meteorite black', 'burgundy red', 'sapphire black', 'crystal white', 'amethyst gray', 'royal blue', 'grey', 'neon orange', 'midnight black', 'mineral black', 'forest', 'champagne white', 'royal gold', 'champagne gold', 'brown', 'champagne gold', 'glossy black', 'cherry gold', 'jazz gold', 'mocha gold', 'rose gold', 'pearl white', 'quite black', 'amazing silver', 'brilliant black', 'stone black', 'black gold', 'gunmetal gray', 'carbon gray', 'dazzling gold', 'meteor gray', 'meteor grey', 'marble white', 'white birch', 'glacier blue', 'meridian gray', 'indigo black', 'santorini white', 'polar white', 'vanilla white', 'glacial silver', 'violet', 'radical yellow', 'stealth black', 'california blue', 'domino', 'active black', 'clearwater', 'active white', 'brown (teflon coating)', 'sophisticated black', 'metal grey', 'amber gold', 'aurora blue', 'ceramic white', 'mystic silver', 'golden black', 'haze gold', 'graphite black', 'agate red', 'pink', 'prestige gold', 'obsidian black', 'moonlight silver', 'aluminium', 'luxurious gold', 'moonlight silver', 'obsidian black', 'snow white', 'crystal black', 'metallic black', 'crystal blue', 'matt black', 'shadow black', 'black body', 'orange', 'deep blue', 'icy white', 'dark blue', 'dark grey', 'metallic brown', 'aurora black', 'onyx black', 'gunmetal grey', 'matte black', 'platinum', 'matte white', 'tuxedo black', 'crimson', 'ebony', 'golden', 'titanium', 'flamingo red', 'mystic white', 'titan', 'indigo', 'titan silver', 'metallic black', 'space black', 'carbon', 'brilliant bronze', 'navy', 'blue', 'metallic gray', 'metallic silver', 'platinum silver', 'silk white', 'blue (boost mobile)', 'titan silver', 'purple', 'black & silver', 'champanage gold', 'mint green', 'black (front panel)', 'black front & white back', 'black (front)', 'white champagne', 'magnetic black', 'classy silver', 'cocoa brown', 'matte cyan', 'glossy cyan', 'iron gray', 'iron grey', 'licorice black', 'lunar gray', 'metallic cherry', 'lunar grey', 'black ballistic nylon', 'deep sea blue', 'black lens', 'champagne', 'licorice', 'cloud gray', 'electric magic black', 'golden bronze', 'black slate', 'midnight titanium', 'arte black', 'bright orange', 'glossy green', 'bright orange', 'bright green', 'cyan', 'lime green', 'matte black', 'steel dark', 'cool grey', 'nokia oro light', 'dark steel', 'frosty metal', 'silver grey', 'charcoal black', 'azure', 'carbon black', 'blue on white', 'chrome', 'white steel', 'black and cyan', 'gunmetal', 'graphite', 'onyx', 'sandstone black', 'obsidian', 'marine blue', 'steel blue', 'black sky', 'blue coral', 'camo green', 'black onyx', 'black sapphire', 'white frost', 'smoky titanium', 'white pearl', 'frosted white', 'titanium green', 'iris charcoal', 'electric blue', 'dazzling white', 'pebble blue', 'cream white', 'dive blue', 'black and gold', 'dark brown', 'luxury brown', 'urban grey', 'metallic blue', 'chic white', 'metallic gray and white', 'navy blue', 'carbon blue', 'noble black', 'metallic gray', 'black and grey', 'deep black', 'black and bronze', 'modern black', 'hanayome white', 'snowy white', 'luminous chrome', 'mineral black', 'serene black', 'classic black', 'blue gradient', 'iconic black', 'sensous black', 'elegant black', 'solid black', 'dark ale', 'sunburst', 'dark gray metallic', 'jet calf', 'onyx calf', 'jet', 'crown gold', 'ceramic black', 'coffee brown', 'moonstone grey', 'ion gold']
brand_arr=['xioami','apple','vodafone','iphone','redmi','zenfone','oppo','vivo','nokia','samsung','allview','htc','gionee','oneplus','accer','acer','sonim','qmobile','panasonic','verykool','huawei','asus','lenovo','coolpad','alcatelis','jio']
model_arr=['model aa', 'model b', 'model ff', 'model d', 'sure , model a', 'model h', 'model j', 'model ff', 'model j', 'model e', 'model k', 'model h', 'model h', 'model e', 'model b', 'model ff', 'model a', 'model ff', 'xplay6', 'iphone 7 plus', 'desire 10 pro', 'zuk z2 pro', 'model lumia 950', 'dtek60', 'moto maxx', 'liquid x2', 'galaxy j7 ( 2017 )', 'one max', 'a1', 'u ultra', 'pixel v2', 'mate 9 porsche design', 'liquid z520', 'one x10', 'galaxy c7 pro', 'x4 soul', 'galaxy c7 pro', 'redmi note prime', 'razr d1', 'zenfone 3 zoom ze553kl', 'moto maxx', 'f3 plus', 'galaxy c9 pro', 'r9s plus', 'f3', 'honor mx6', 'sure , v5s', 'xperia x performance', 'liquid jade primo', 'phone ms1', 'r11', 'k8 ( 2017 )', 'n3', 'galaxy c9 pro', 'v5s', 'xperia z4 tablet lte', '3t', 'galaxy j7 max', 'x9', 'v5s', 'a57', 'moto maxx', 'a57', 'galaxy c9 pro', 'x5max platinum edition', 'model xperia xa1 ultra', 'xperia xz premium', '2 3g', 'xperia xz premium', 'galaxy c7 pro', 'xperia xz', 'liquid jade primo', 'xperia l1', 'ms1', 'xperia xa1 ultra', 'mate 9', 'model desire 10 pro', 'xperia xz premium', 'mate 9', 'liquid x2', 'one e9+', 'one x10', 'marathon m5 plus', 'w909', 'zenfone live zb501kl', 'mate 9', 'axon lux', 'honor 7', 'galaxy a5 duos', 'a1', 'iconia talk s', 'model galaxy j3 emerge', 'liquid jade primo', 'note 5 lite', 'xperia xa ultra', 'xperia e4 dual', 'vibe p1m', 'xplay6', 'blade s7', 'x4 soul', 'zenfone 3s max zc521tl', 'pop 7 lte', 'dtek60', 'model nova 2', 'a59', 'r9 plus', 'model galaxy c9 pro', 'satio ( idou )', 'model mi note 2', 'galaxy j7 max', 'model galaxy c9 pro', 'galaxy s5 neo', 'model moto e3 power', 'droid turbo', 'model f1', 'model galaxy c7', 'r11 plus', 'signature touch ( 2015 )', 'then elife e8', 'mi 5c', 'blade a2 plus', 'galaxy j7 max', 'x7', 'a6600 plus', 'g6', 'm3 max', 'liquid x2', 'galaxy j3 ( 2017 )', 'one x10', 'hero cdma', 'moto x play dual sim', 'u11', '5', 'elife e8', 'zenfone 5 a500kl', 'iphone 7 plus', 'honor note 8', 'sure , g6', 'aurora', 's10', 's10', 'desire 830', 'galaxy s8', 'galaxy j7 ( 2016 )', 'v5 plus', 'redmi 3 pro', 'blade v8', 'axon 7 max', 'mi note 2', 'galaxy c9 pro', 'noir z8', 'xperia xa1 ultra', 'galaxy s7 active', "porsche design p'9982", '5', 'zenfone ar zs571kl', 'zenfone 3s max zc521tl', 'honor 9', 'nova 2', 'leap', 's10', 's9', 'pixel', 'moto e4 plus', 'eluga i3 mega', 'galaxy c7 pro', 'v3max', 'model y67', 'y67', 'idol 4s', 'blade a2 plus', 'a37', 'galaxy note7', 'galaxy a7 ( 2017 )', 'xperia c5 ultra dual', 'xperia xa dual', 'sure , model redmi note 4', 'mi 6', 'xplay5', 'z2', 'ms1', 'f1', 'f3 plus', 'liquid x2', 'zenfone 3 max zc553kl', 'lumia 630 dual sim', 'find 7a', 'galaxy s8+', 'galaxy s7', 'eluga ray x', 'xperia xz premium', 'mi note pro', 'iphone 6 plus', 'zenfone 3 laser zc551kl', 'z30', 'model butterfly 3', 'sure , f3', 'galaxy xcover 4', 'u ultra', 'one m9s', 'k6', 'g6', 'r7 plus', 'iconia talk s', 'x6s', 'blade qlux 4g', 'axon lux', 'noir e8', 'model zenfone live zb501kl', 'dtek50', 'nowadays s9', 'e7', 'model lumia 640 xl lte dual sim', 'model iphone 6s', 'sure , x5max platinum edition', 'model aurora', 'okay highway', 'ipaq hw6515', 'one m8s', 'desire 626g+', 'model honor 9', "model porsche design p'9983", 'r11', 'model moto e dual sim ( 2nd gen )', 'mc55', 'well , then v5s', 'model redmi 3 pro', 'p8 lite ( 2017 )', '6', 'galaxy xcover 4', 'sure , x9', 'moto e3 power', 'm2017', 'marathon m4', 'xperia z2a', 'zenfone 6 a601cg', 'aurora', 'iphone 7', 'galaxy a7 ( 2017 )', 'xperia e4g dual', 'xperia e5', 'xperia x', 'have a look at f5', 'sure , moto z2 play', 'iphone 7', 'conjr', 'fierce xl', 'p8 energy pro', 'v2 viper', 'blade v8', 'yutopia', '5 zs550kl', 'zenfone live zb501kl', 'a1 plus', 'cool s1', 'v10', 'r7 plus', 'gsmart classic lite', '1', 'k20 plus', 'mytouch q 2', 'canvas sliver 5 q450', 'f1 plus', 'xperia l1', '10 evo', 'honor 8 pro', 'e5', 'galaxy j2 prime', 'zenfone 2 deluxe ze551ml', 'marathon m5 plus', 'e6', 'eluga switch', 'm6s plus', 'sure , y7', 'sure , r3', 'moto z2 play', 'moto e4 plus ( usa )', 'canvas tab p666', 'p10 plus', 'le max 2', 'smart platinum 7', 'x7 plus', 'padfone infinity 2', 'a1 plus', 'x', 'eluga pulse', 'signature touch ( 2015 )', 'model v3', 'r9s', 'iphone 5s', 'pure xl', 'one ( m8 ) for windows', 'mate 9 pro', 'honor 3x pro', 'x10', 'model galaxy j1 ( 2016 )', 'galaxy c9 pro', 'honor magic', 'moto z force', 'light weight moto x ( 2nd gen )', 'one u11', 'iphone 7 plus', 'iphone 7', 'z30', 'zenfone go zb500kl', 'zenpad 3s 10 z500kl', '5 zs550kl', 'vivo iv', 'galaxy a7 ( 2017 )', 'lumia 2520', 'dtek50', 'galaxy c7', 'sure , w909', 'slate6 voicetab', 'model galaxy j7 prime', 'x venture', 'y28', 'blade s7', 'v5', 'tab prime 6', '3t', 'f3', 'one x10', 'x7', 'v3', 'xperia xa1 ultra', 'x6s', 'model xperia xa dual', 'xperia xa1', 'f3 plus', 'iphone 6s plus', 'model fire c', 'model zenfone ar zs571kl', 'priv', 'model iphone 7', 'galaxy s8', 'model s6 pro', 's9', 'model pioneer p6', 'model honor 8', 'model w909', 'blade v8', 'model blade g', 'iphone 6s', 'galaxy s8', 'model galaxy a5 ( 2017 )', 'x2 soul', 'mediapad m3 lite 10', '3', 's5015 spark ii', 'galaxy j3 emerge', 'galaxy note7', 'noir e8', 'galaxy xcover 4', 'mytouch q', 'canvas pulse 4g e451', 'pure xl', 's6', 'mi max 2', 'redmi 3 pro', 'redmi 3s', 'yutopia', 'pro 6 plus', 'iphone 5', '5 )', 'x3 soul pro', 'zenfone 5 a500kl', 'galaxy j7 max', 'galaxy j1 ace', 'galaxy j1 ace', 'galaxy s6 edge+', 'desire 728 dual sim', 'one m9s', 'a57', 'f1', 'eluga switch', 'pixel xl', 'one x10', 'z25', 'liquid z6 plus', 's9', 'honor 5a', 'model yoga tab 3 plus', 'u10', 'noir z8', 'xperia xa ultra', 'xp7', 'sl6010 cyprus lte', 'galaxy s8+']
ref_arr=['2012-','2012_','2020_','2020-','1212_','1212-'] 
radio_arr=['yes','no'] 
sim_arr=['dual','single','multiple','yes','no'] 
dualsim_arr=['yes','no']
gps_arr=['yes','no']
quarter_to_month={'q4.released2009q4':'december','q1':'march','q2':'june','q3':'september','q4':'december','1q':'march','2q':'june','3q':'september','4q':'december'}

In [78]:
#word_to_id['ram']

In [10]:
print(padd('hey, how are you?'))
def sent_to_emb(sent): 
    sent=padd(sent)
    emb_arr=[] 
    for i in range(15):
        if sent[i] in word_to_emb:
            emb_arr.append(word_to_emb[sent[i]])
        else:
            emb_arr.append(word_to_emb['<unk>'])
    return emb_arr

['hey', ',', 'how', 'are', 'you', '?', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


In [11]:
vocab_f=open('data/glove/glove.6B.50d.txt',encoding='utf8')
word_to_id={} 
word_to_emb={}
val=4
word_to_id['<pad>']=3 
word_to_id['<unk>']=0
word_to_id['<start>']=1
word_to_id['<end>']=2
for line in vocab_f:
  word_to_id[line.split()[0]]=val 
  word_to_emb[line.split()[0]]=[float(i) for i in line.split()[1:]]  
  val+=1 
print(word_to_id['20']) 
print(len(word_to_emb['happy']))

328
50


In [12]:
word_to_emb['<unk>']=[0]*50 
word_to_emb['<pad>']=[0]*50 
temp_agent=[word_to_emb['<unk>']]  
import pandas as pd
#df=pd.read_excel('data/SharedData.xlsx')
xl=pd.ExcelFile('data/database.xlsx')
df=pd.DataFrame() 
df=xl.parse(xl.sheet_names[0])
User=[] 
User_Arr=[]
for row in df['User']:
    if type(row)==int:
        User.append([]) 
    else:
        User[-1].append(row.lower()) 
        User_Arr.append(row.lower()) 
Agent=[]
Agent_Arr=[]
for row in df['Agent']:
    if type(row)==int:
        Agent.append([]) 
    else:
        Agent[-1].append(row.lower()) 
        Agent_Arr.append(row.lower()) 
# Intent=[]
# Intent_Arr=[]
# for row in df['Intent']:
#     if type(row)==int:
#         Intent.append([]) 
#     else:
#         Intent[-1].append(row.lower()) 
#         Intent_Arr.append(row.lower()) 
Slot=[]
Slot_Arr=[]
Value=[]
Value_Arr=[]
temp_slot=[]
temp_val=[]
is_slot=[]
cnt=0
for row in df['Slot']:
    if type(row)==int:
        Slot.append([]) 
        Value.append([]) 
        #temp_slot=[]
        #temp_val=[]
    elif type(row)==float:
        Slot[-1].append('null') 
        Value[-1].append('null')
        Slot_Arr.append(['null']) 
        Value_Arr.append(['null']) 
        temp_slot=['null'] 
        temp_val=['null']
    else: 
        temp_slot=[] 
        temp_val=[] 
        print(row)
        part=row.split(',') 
        for p in part:
          if p=='':
            p='null-null'
          Slot[-1].append(correct[p.split('-')[0]]) 
          temp_slot.append(correct[p.split('-')[0]]) 
          if len(p.split('-'))>1 and p.split('-')[1].lower() in val_mp:
            Value[-1].append(val_mp[p.split('-')[1].lower()]) 
            temp_val.append(val_mp[p.split('-')[1].lower()])
            if temp_val[-1]=='t':
                temp_val[-1]='t-mobile'
          elif len(p.split('-'))>1:
            Value[-1].append(p.split('-')[1].lower())
            temp_val.append(p.split('-')[1].lower())
          else:
            Value[-1].append('null') 
            temp_val.append('null') 
    if type(row)!=int and type(row)!=float:
      Slot_Arr.append([]) 
      for a in temp_slot:
        Slot_Arr[-1].append(a)  
      Value_Arr.append([]) 
      for a in temp_val:
        Value_Arr[-1].append(a)    
A_Action=[]
A_Action_Arr=[]
for row in df['A_Action']:
    if type(row)==int:
        A_Action.append([])
    elif type(row)==float:
        A_Action[-1].append('null') 
    else:
        row=row.split(',')[0]
        A_Action[-1].append(a_action_mp[row]) 
        A_Action_Arr.append(a_action_mp[row]) 
U_Action=[]
U_Action_Arr=[]
for row in df['U_Action']:
    if type(row)==int:
        U_Action.append([])
    elif type(row)==float:
        U_Action[-1].append('null') 
        U_Action_Arr.append('null')  
    else:
        U_Action[-1].append(u_action_mp[row]) 
        U_Action_Arr.append(u_action_mp[row]) 

Camera-Good,Battery-Good
Color-Black
OS-Android,Sim-Dual
Color-White
Touch-Y,Memory-Good,Camera-Good
OS-Android
Color-Grey,Memory-Good
Cost-less
Camera-Good,Rating-High
Cost-Less
Camera-Good
Memory-High
Camera-Good
Memory-Good
Cost-Less
Touch-Y,Battery-Good
Price-15k-
Color-White,Memory-Good
Price-10k-
Released_Yr-Latest
OS-Non-Windows
OS-Android
Touch-No
Cost-Less
Company-Vivo,Model-I
Color-Gold
Camera-15+,Color-Gold
Camera-Good,Memory-Good,OS-Windows
Total_Review-15K+
Sim-Dual
Touch-Y
Cost-Less
Company-Nokia,Model-E
Cost-Less
Total_Review-16k+,Battery-Good
Price-20k-
OS-Android
Rating-Good,Released_Yr-2019
Color-Black
Touch-No
Cost-2k-
Sim-Dual,Cost-Less
Memory-6GB
Camera-15
OS-Android
Camera-Good,Rating-Good
Cost-Less
Sim-Dual
Cost-Less
Company-Jio,Model-F
Cost-Less
OS-Android
Rating-Good
OS-Android
Color-Blue,Sim-Dual
Cost-15k-
Rating-4+
Battery-Good,RAM-Good
Color-Silver
Battery-Good,Display_Size-Big
Weight_g-Light
Brand-HTC,Model-Paradise
Color-Blue
Brand-Lenovo,Model-Yoga Tab 3


In [13]:
df=pd.read_csv('data/final.csv')
sentiment_inp=[]
sentiment_label=[] 
for line in df['Utterance']:
    sentiment_inp.append(sent_to_emb(line)) 
for line in df['Sentiment']:
    if line==-1:
        ans=[1,0,0]
    elif line==0:
        ans=[0,1,0] 
    else:
        ans=[0,0,1] 
    sentiment_label.append(ans) 
print(len(sentiment_inp),len(sentiment_label)) 

8054 8054


In [14]:
print(len(User_Arr),len(Agent_Arr),len(Slot_Arr),len(Value_Arr),len(U_Action_Arr))

4199 4199 4199 4199 4199


In [15]:
#df=pd.read_excel('data/SharedData.xlsx')
xl=pd.ExcelFile('data/database.xlsx')
df=pd.DataFrame() 
df=xl.parse(xl.sheet_names[2])        
for row in df['User']:
    if type(row)==int:
        User.append([]) 
    else:
        User[-1].append(row.lower()) 
        User_Arr.append(row.lower()) 
for row in df['Agent']:
    if type(row)==int:
        Agent.append([]) 
    else: 
        Agent[-1].append(row.lower()) 
        Agent_Arr.append(row.lower()) 
# for row in df['Intent']:
#     if type(row)==int:
#         Intent.append([]) 
#     else:
#         Intent[-1].append(row.lower()) 
#         Intent_Arr.append(row.lower()) 
temp_slot=[]
temp_val=[]
is_slot=[]
cnt=0
for row in df['Slot']:
    #print(row)
    if type(row)==int:
        Slot.append([]) 
        Value.append([]) 
        #temp_slot=[]
        #temp_val=[]
    elif type(row)==float:
        Slot[-1].append('null') 
        Value[-1].append('null')
        Slot_Arr.append(['null']) 
        Value_Arr.append(['null']) 
        temp_slot=['null'] 
        temp_val=['null']
    else: 
        temp_slot=[] 
        temp_val=[] 
        part=row.split(',') 
        for p in part: 
          Slot[-1].append(correct[p.split('-')[0]]) 
          temp_slot.append(correct[p.split('-')[0]]) 
          if len(p.split('-'))>1:
            Value[-1].append(val_mp[p.split('-')[1].lower()]) 
            temp_val.append(val_mp[p.split('-')[1].lower()])
            if temp_val[-1]=='t':
                temp_val[-1]='t-mobile'
          else:
            Value[-1].append('null') 
            temp_val.append('null') 
    if type(row)!=int and type(row)!=float:
      Slot_Arr.append([]) 
      for a in temp_slot:
        Slot_Arr[-1].append(a)  
      Value_Arr.append([]) 
      for a in temp_val:
        Value_Arr[-1].append(a)    
# for row in df['A_Action']:
#     if type(row)==int:
#         A_Action.append([])
#     elif type(row)==float:
#         A_Action[-1].append('null') 
#     else:
#         row=row.split(',')[0]
#         A_Action[-1].append(a_action_mp[row]) 
#         A_Action_Arr.append(a_action_mp[row]) 
temp_mp={}
for row in df['U_Action']:
    if type(row)==int:
        U_Action.append([])
    elif type(row)==float:
        U_Action[-1].append('null') 
        U_Action_Arr.append('null')  
    else:
        U_Action[-1].append(u_action_mp[row]) 
        U_Action_Arr.append(u_action_mp[row]) 

In [16]:
print(len(User_Arr),len(Agent_Arr),len(Slot_Arr),len(Value_Arr),len(U_Action_Arr))

8384 8384 8384 8384 8384


In [17]:
import pandas as pd
xl=pd.ExcelFile('data/database.xlsx')
df=pd.DataFrame() 
df=xl.parse(xl.sheet_names[1])
db=[]
for i in range(0,len(df['Serial'])):
  j=i
  db.append({'ref_no':'','cost_price':'','money_unit':'','month_name':'','year_name':'','review_person':'3500','rating_score':'4.4','cam_val':'','mem_size':'','bat_size':'','color_name':'','os_name':'','model_name':'','brand_name':''})
  db[j]['ref_no']=str(j)
  for k in df.keys():
    if k=='brand':
      db[j]['brand_name']=str(df[k][j]).lower() if str(df[k][j]).lower() not in val_mp else val_mp[str(df[k][j]).lower()]  
    if k=='model':
      db[j]['model_name']=str(df[k][j]).lower() if str(df[k][j]).lower() not in val_mp else val_mp[str(df[k][j]).lower()] 
    if k=='Battery':
      if df[k][j]<100:
        df[k][j]*=100 
      if df[k][j]>50000:
        df[k][j]=df[k][j]%1000
      db[j]['bat_size']=str(df[k][j]) if str(df[k][j]) not in val_mp else val_mp[df[k][j]] 
    if k=='P_Camera':
      db[j]['cam_val']=str(df[k][j]).lower() if str(df[k][j]).lower() not in val_mp else val_mp[str(df[k][j]).lower()] 
      if type(df[k][j])==str and 'mp' in df[k][j].lower():
            db[j]['cam_val']=df[k][j][:-2]
    if k=='OS':
      db[j]['os_name']=str(df[k][j]).lower() if str(df[k][j]).lower() not in val_mp else val_mp[str(df[k][j]).lower()]  
    if k=='Color':
      db[j]['color_name']=str(df[k][j]).lower() if str(df[k][j]).lower() not in val_mp else val_mp[str(df[k][j]).lower()]  
    if k=='Internal_RAM':
      db[j]['mem_size']=str(df[k][j]).lower() if str(df[k][j]).lower() not in val_mp else val_mp[str(df[k][j]).lower()]  
    if k=='approx_price_EUR':
      db[j]['cost_price']=str(df[k][j]).lower() if str(df[k][j]).lower() not in val_mp else val_mp[str(df[k][j]).lower()]  
      db[j]['money_unit']='euro' 
    if k=='ReleasedYear':
      db[j]['year_name']=str(df[k][j]).lower() if str(df[k][j]).lower() not in val_mp else val_mp[str(df[k][j]).lower()]  
    if k=='ReleasedMonth':
      db[j]['month_name']=str(df[k][j]).lower() if str(df[k][j]).lower() not in val_mp else val_mp[str(df[k][j]).lower()] 
      month_flag=0
      for month in month_arr:
        if month in db[j]['month_name']:
            db[j]['month_name']=month
            month_flag=1 
            break 
      if month_flag==0:
        db[j]['month_name']=quarter_to_month[db[j]['month_name']]
    if k=='weight_g':
      db[j]['weight_val']=str(df[k][j]).lower() if str(df[k][j]).lower() not in val_mp else val_mp[str(df[k][j]).lower()] 
    if k=='Radio':
      if df[k][j]=='Y':
        tmp='yes' 
      else:
        tmp='no'
      db[j]['rad_bin']=tmp 
    if k=='Display_Size':
      db[j]['size_val']=str(df[k][j]).lower() if str(df[k][j]).lower() not in val_mp else val_mp[str(df[k][j]).lower()]  
    if k=='GPS':
      db[j]['gps_present']='yes' if df[k][j]=='Y' else 'no'
    if k=='SIM':
      db[j]['sim_number']='single' if df[k][j]=='S' or df[k][j]=='N' else 'dual' if df[k][j]=='D' else 'multiple'
    if k=='img_url':
        db[j]['img_url']=str(df[k][j])
temp_col_mp={} 
common_color=['black','white','red','blue','gold','silver','pink','orange','titanium','cyan','chrome','charcoal','bronze','gunmetal','gray','grey','yellow','brown','dark','calf','light','green','metallic','frosty','clearwater','platinum','aluminium']
for data in db:
    if data['color_name'] not in temp_col_mp:
        ans=data['color_name'] 
        for col in common_color:
            if col in ans:
                ans=col 
                break
        if ans=='ardesia' or ans=='allview':
            ans='slate'
        temp_col_mp[data['color_name']]=ans 
len(db)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


2517

In [18]:

word_to_emb['allview']=[((float(word_to_emb['acer'][i])+float(word_to_emb['asus'][i]))/2) for i in range(50)]
word_to_emb['celkon']=[((float(word_to_emb['zte'][i])+float(word_to_emb['meizu'][i]))/2) for i in range(50)]
word_to_emb['coolpad']=[((float(word_to_emb['vivo'][i])+float(word_to_emb['oppo'][i]))/2) for i in range(50)]
word_to_emb['garmin-asus']=word_to_emb['asus']
word_to_emb['gionee']=[((float(word_to_emb['xiaomi'][i])+float(word_to_emb['vivo'][i]))/2) for i in range(50)] 
word_to_emb['yezz']=[((float(word_to_emb['zte'][i])+float(word_to_emb['pantech'][i]))/2) for i in range(50)]
word_to_emb['i-mate']=[((float(word_to_emb['kyocera'][i])+float(word_to_emb['jolla'][i]))/2) for i in range(50)]
word_to_emb['intex']=[((float(word_to_emb['micromax'][i])+float(word_to_emb['lava'][i]))/2) for i in range(50)]
word_to_emb['leeco']=[((float(word_to_emb['intex'][i])+float(word_to_emb['i-mate'][i]))/2) for i in range(50)]
word_to_emb['oneplus']=[((float(word_to_emb['apple'][i])+float(word_to_emb['samsung'][i]))/2) for i in range(50)]
word_to_emb['karbonn']=word_to_emb['karbon']
word_to_emb['prestigio']=[((float(word_to_emb['apple'][i])+float(word_to_emb['asus'][i]))/2) for i in range(50)]
word_to_emb['qmobile']=[((float(word_to_emb['nokia'][i])+float(word_to_emb['oppo'][i]))/2) for i in range(50)]
word_to_emb['verykool']=[((float(word_to_emb['coolpad'][i])+float(word_to_emb['lg'][i]))/2) for i in range(50)]
word_to_emb['wiko']=[((float(word_to_emb['oppo'][i])+float(word_to_emb['asus'][i]))/2) for i in range(50)]
word_to_emb['sonim']=[((float(word_to_emb['sony'][i])+float(word_to_emb['samsung'][i]))/2) for i in range(50)]
word_to_emb['sony ericsson']=word_to_emb['ericsson'] 
word_to_emb['redmi']=word_to_emb['xiaomi'] 
word_to_emb['dualsim']=word_to_emb['sim']
word_to_emb['releasedyear']=word_to_emb['year']
word_to_emb['releasedmonth']=word_to_emb['month']
for i in db[0]:
  word_to_emb[i]=word_to_emb['value']

tmp_mp={}
cnt=0
for i in range(len(Slot_Arr)):
    for j in range(len(Slot_Arr[i])):
        if Slot_Arr[i][j]=='brand' and Value_Arr[i][j] not in tmp_mp:
            tmp_mp[Value_Arr[i][j]]=cnt
            cnt+=1


In [19]:
db_out=[] 
db_in=[]
cnt_temp=0
key_to_pos={} 
for data in db:
    db_out.append([]) 
    for i in range(2517):
        if i==cnt_temp:
            db_out[-1].append(1) 
        else:
            db_out[-1].append(0) 
    cnt_temp+=1
    db_in.append([])
    db_in[-1].append(word_to_emb[str(data['brand_name'])]) 
    key_to_pos['brand']=0
    if float(data['cost_price'])>1000:
        data['cost_price']=(int(float(data['cost_price']))//100)*100
    if float(data['cost_price'])>10000:
        data['cost_price']=(int(float(data['cost_price']))//100)
    db_in[-1].append(word_to_emb[str(data['cost_price'])])
    key_to_pos['cost']=1
    db_in[-1].append(word_to_emb[str(data['month_name'])])
    key_to_pos['month']=2
    db_in[-1].append(word_to_emb[str(data['year_name'])])
    key_to_pos['year']=3
    db_in[-1].append(word_to_emb[str(data['cam_val']).lower()]) 
    key_to_pos['camera']=4
    if type(data['mem_size'])==str and ('GB' in data['mem_size'] or 'gb' in data['mem_size']):
        data['mem_size']=data['mem_size'][:-2]
    db_in[-1].append(word_to_emb[str(int(float(data['mem_size']))//1)])
    key_to_pos['memory']=5
    key_to_pos['ram']=5
    temp_sz=isNan(data['bat_size'])
    key_to_pos['battery']=6
    if temp_sz>10000:
        temp_sz=(temp_sz//1000)*1000
    db_in[-1].append(word_to_emb[str((temp_sz//100)*100)])
    db_in[-1].append(word_to_emb[str(temp_col_mp[data['color_name']])])
    key_to_pos['color']=7
    db_in[-1].append(word_to_emb[str(data['os_name'])])
    key_to_pos['os']=8
    db_in[-1].append(word_to_emb[str(data['rad_bin'])])
    key_to_pos['radio']=9
    db_in[-1].append(word_to_emb[str(data['size_val'])])
    key_to_pos['display']=10
    if type(data['weight_val'])==str:
        data['weight_val']=data['weight_val'].split('-')[0]
    key_to_pos['weight']=11
    db_in[-1].append(word_to_emb[str(int(float(data['weight_val']))//1)]) 
    key_to_pos['sim']=12 
    db_in[-1].append(word_to_emb[data['sim_number']])
    key_to_pos['gps']=13 
    db_in[-1].append(word_to_emb[data['gps_present']])
print(len(db_in[0]))

14


In [20]:
import math
for d in db:
    if not math.isnan(float(d['bat_size'])):
        d['bat_size']=str(int(float(d['bat_size'])))

In [21]:
nltk.download('punkt')
def padd(string):
  arr=word_tokenize(string)  
  l=len(arr) 
  for i in range(max(0,15-l)):
    arr.append('<pad>') 
  return arr[:15]  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [22]:
temp=[]
for m in range(2000,2020):
  temp.append(str(m)) 
print(temp)

['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']


In [23]:
for i in range(len(User_Arr)):
  a=word_tokenize(Agent_Arr[i])
  for j in range(len(a)):
    st=a[j]
    #print(st)
    for k in range(len(a[j])):
      if a[j][k]=='.' and len(a[j])!=1 and a[j][k-1]!=' ':
        st=a[j][:k]+' '+'.'+' '+a[j][k+1:] 
        break 
    if 'mah' in st and len(st)>3:
      st=st[:-3]+' '+st[-3:] 
    for r in ref_arr:
      if r in st: 
        st='ref_no'
        break
    if st in clean_mp:
      st=clean_mp[st]  
    a[j]=st 
  Agent_Arr[i]=' '.join(a)
print(Agent_Arr[i])

welcome .


In [24]:
db_dummy=[] 
org_Agent_Arr=[] 
for i in range(len(User_Arr)):
  org_Agent_Arr.append(Agent_Arr[i]) 
for i in range(len(User_Arr)):
  if U_Action_Arr[i]=='rec_slt' or U_Action_Arr[i]=='feedback' or U_Action_Arr[i]=='confirm':
    for model in model_arr:
      if model in Agent_Arr[i]:
          Agent_Arr[i]=Agent_Arr[i].replace(model,'model_name')
    for dd in db:
      brand=dd['brand_name']
      Agent_Arr[i]=Agent_Arr[i].replace(brand,'brand_name') 
    for col in color_arr:
       Agent_Arr[i]=Agent_Arr[i].replace(col,'color_name')
  if U_Action_Arr[i]=='battery':
    for bat in bat_arr:
      Agent_Arr[i]=Agent_Arr[i].replace(bat,'bat_size') 
  if U_Action_Arr[i]=='camera':
    for cam in cam_arr:
      Agent_Arr[i]=Agent_Arr[i].replace(cam,'cam_val') 
  if U_Action_Arr[i]=='color':
    for color in color_arr:
      Agent_Arr[i]=Agent_Arr[i].replace(color,'color_name') 
  if U_Action_Arr[i]=='os':
    for os in os_arr:
      Agent_Arr[i]=Agent_Arr[i].replace(os,'os_name') 
  if U_Action_Arr[i]=='memory':
    for mem in mem_arr:
      Agent_Arr[i]=Agent_Arr[i].replace(mem,'mem_size') 
  if U_Action_Arr[i]=='ram':
    for mem in mem_arr:
      Agent_Arr[i]=Agent_Arr[i].replace(mem,'mem_size') 
  if U_Action_Arr[i]=='specification' or U_Action_Arr[i]=='feedback':
    a=word_tokenize(Agent_Arr[i])
    for j in range(len(a)):
      st=a[j] 
      if st=='mp':
        a[j-1]='cam_val' 
      elif st=='gb':
        a[j-1]='mem_size' 
      elif st=='mah':
        a[j-1]='bat_size' 
    Agent_Arr[i]=' '.join(a) 
  if U_Action_Arr[i]=='review' or U_Action_Arr[i]=='rating':
    for r in rating_arr:
      Agent_Arr[i]=Agent_Arr[i].replace(r,'rating_score') 
    a=word_tokenize(Agent_Arr[i]) 
    for j in range(len(a)):
      if isint(a[j])==1 and int(a[j])>1000:
        a[j]='review_person' 
    Agent_Arr[i]=' '.join(a) 
  if U_Action_Arr[i]=='month' or U_Action_Arr[i]=='year':
    for m in month_arr:
      Agent_Arr[i]=Agent_Arr[i].replace(m,'month_name') 
    for y in year_arr:
      Agent_Arr[i]=Agent_Arr[i].replace(y,'year_name') 
  if U_Action_Arr[i]=='weight':
    a=word_tokenize(Agent_Arr[i]) 
    for j in range(len(a)):
      if isint(a[j])==1:
        a[j]='weight_val'
        break 
    Agent_Arr[i]=' '.join(a) 
  if U_Action_Arr[i]=='size':
    a=word_tokenize(Agent_Arr[i]) 
    for j in range(len(a)):
      if isint(a[j])==1:
        a[j]='size_val' 
        break
    Agent_Arr[i]=' '.join(a) 
  if U_Action_Arr[i]=='display':
    a=word_tokenize(Agent_Arr[i]) 
    for j in range(len(a)):
      if isint(a[j])==1:
        a[j]='size_val' 
        break
    Agent_Arr[i]=' '.join(a) 
  if U_Action_Arr[i]=='sim' or U_Action_Arr=='dualsim':
    for v in sim_arr:
        Agent_Arr[i]=Agent_Arr[i].replace(v,'sim_number') 
  if U_Action_Arr[i]=='radio':
    for v in radio_arr:
        Agent_Arr[i]=Agent_Arr[i].replace(v,'rad_bin')
  if U_Action_Arr[i]=='gps':
    for v in gps_arr:
        Agent_Arr[i]=Agent_Arr[i].replace(v,'gps_present')
  if U_Action_Arr[i]=='cost':
    a=word_tokenize(Agent_Arr[i]) 
    for j in range(len(a)):
      if isint(a[j])==1 and int(a[j])>100:
        a[j]='cost_price' 
        break
    Agent_Arr[i]=' '.join(a)
    for u in ['euro','rs','rupees']:
      Agent_Arr[i]=Agent_Arr[i].replace(u,'money_unit') 
  #print(U_Action_Arr[i],Agent_Arr[i])  
print(Agent_Arr[1]) 
print(org_Agent_Arr[1])

model_name by brand_name is one of the best phone for camera as well as battery
model aa by oppo is one of the best phone for camera as well as battery


In [25]:
for i in range(len(Agent_Arr)):
    if Agent_Arr[i]!=org_Agent_Arr[i]:
        print(i,Agent_Arr[i],org_Agent_Arr[i])

1 model_name by brand_name is one of the best phone for camera as well as battery model aa by oppo is one of the best phone for camera as well as battery
2 its cost is money_unit cost_price only its cost is rs 9999 only
3 the color of oppo aa is color_name the color of oppo aa is white
4 sir this model_nameas strong features but available in color_name only . sir this model has strong features but available in white only .
5 you can take model_name by color_namemi you can take model a by redmi
6 it 's of cost_price . it 's of 7600 .
10 model_name by brand_name is one of the best phone suited to your need . model b by samsung is one of the best phone suited to your need .
11 it cost money_unit cost_price . it cost rs 6800 .
12 the color of the phone is color_name the color of the phone is gold
13 this model is available in color_name only . color_name is trending these days this model is available in gold only . gold is trending these days
14 you can go for aa model_namey brand_name you

In [26]:
name_to_slot={'ref_no':'reference','cost_price':'cost','money_unit':'money','month_name':'month','year_name':'year','review_person':'review','rating_score':'rating','cam_val':'camera','mem_size':'memory','bat_size':'battery','color_name':'color','os_name':'os','model_name':'model','brand_name':'brand','weight_val':'weight','size_val':'size','sim_number':'sim','gps_present':'gps','rad_bin':'radio'} 
slot_to_key={} 
for k,v in name_to_slot.items():
  if v not in slot_to_key:
    slot_to_key[v]=k 
slot_to_key['ram']='mem_size'
slot_to_key['display']='size_val'
slot_to_key['rec_slt']='model_name' 
#slot_to_key

In [27]:
import nltk
nltk.download('punkt')
for i in range(len(User_Arr)):
  a=word_tokenize(User_Arr[i])
  for j in range(len(a)):
    st=a[j]
    #print(st)
    for k in range(len(a[j])):
      if a[j][k]=='.':
        st=a[j][:k]+' '+'.'+' '+a[j][k+1:] 
        break 
    a[j]=st 
  User_Arr[i]=' '.join(a)
print(User_Arr[i])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
thanks for help  . 


In [28]:
from nltk.corpus import stopwords
stop_words=set(stopwords.words('english'))
user_arr=[]
for dial in User_Arr:
#   temp=padd(dial) 
#   for i in temp:
#         if i in stop_words:
#             temp.remove(i) 
#   dial=' '.join(temp) 
  user_arr.append(padd(dial))  
user_emb=[] 
sm=[0]*50 
for dial in user_arr:
  user_emb.append([])
  for word in dial:
    if word in word_to_emb:
      user_emb[-1].append(word_to_emb[word])
    else:
      user_emb[-1].append(word_to_emb['<unk>']) 
print(len(user_emb)) 

8384


In [29]:
agent_arr=[]
for dial in Agent_Arr:
  agent_arr.append(padd(dial))  
agent_emb=[] 
for dial in agent_arr:
  agent_emb.append([]) 
  for word in dial:
    if word in word_to_emb:
      agent_emb[-1].append(word_to_emb[word])
    else:
      agent_emb[-1].append(word_to_emb['<unk>']) 
#print(agent_emb[1])
print(len(agent_emb)) 

8384


In [30]:
slot=[] 
mp_sl={}
cnt=0
for row in Slot:
  for a in row:
    if a not in mp_sl:
      mp_sl[a]=cnt 
      cnt+=1 
      slot.append(a) 
print(cnt)
slot_arr=[] 
for row in Slot_Arr:
  slot_arr.append([]) 
  for i in range(cnt):
    slot_arr[-1].append(0) 
  for slt in row:
    slot_arr[-1][mp_sl[slt]]=1  
print(len(slot_arr))
print(slot_arr[996])

25
8384
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [31]:
value=[] 
mp_vl={}
cnt=0
for row in Value:
  for a in row:
    if a not in mp_vl:
      mp_vl[a]=cnt 
      cnt+=1 
      value.append(a)
mp_vl['t-mobile']=cnt 
cnt+=1 
print(cnt) 
value_arr=[] 
for row in Value_Arr:
  value_arr.append([]) 
  for i in range(cnt):
    value_arr[-1].append(0) 
  for slt in row:
    value_arr[-1][mp_vl[slt]]=1  
print(len(value_arr))

185
8384


In [32]:
ag_action=[] 
mp_ag={}
cnt=0
for row in A_Action:
  for a in row:
    if a not in mp_ag:
      mp_ag[a]=cnt 
      cnt+=1 
      ag_action.append(a) 
print(cnt)
mp_ag_ag={} 
for k in mp_ag:
  mp_ag_ag[k]=[] 
  for i in range(26):
    if mp_ag[k]==i:
      mp_ag_ag[k].append(1) 
    else:
      mp_ag_ag[k].append(0) 
ag_action_arr=[] 
for row in A_Action:
  for a in row:
    ag_action_arr.append(mp_ag_ag[a]) 
print(len(ag_action_arr))

26
4199


In [33]:
ug_action=[] 
mp_ug={}
cnt=0
for row in U_Action:
  for a in row:
    if a not in mp_ug:
      mp_ug[a]=cnt 
      cnt+=1 
      ug_action.append(a) 
print(cnt)
mp_ug_ug={} 
for k in mp_ug:
  mp_ug_ug[k]=[] 
  for i in range(cnt):
    if mp_ug[k]==i:
      mp_ug_ug[k].append(1) 
    else:
      mp_ug_ug[k].append(0) 
ug_action_arr=[] 
for row in U_Action:
  for a in row:
    ug_action_arr.append(mp_ug_ug[a]) 
print(len(ug_action_arr))
print(mp_ug) 

29
8384
{'phone': 0, 'rec_slt': 1, 'cost': 2, 'color': 3, 'feedback': 4, 'booking': 5, 'thanks': 6, 'dualsim': 7, 'year': 8, 'os': 9, 'camera': 10, 'rating': 11, 'confirm': 12, 'battery': 13, 'memory': 14, 'review': 15, 'specification': 16, 'discount': 17, 'weight': 18, 'display': 19, 'radio': 20, 'ram': 21, 'greet': 22, 'month': 23, 'sim': 24, 'gps': 25, 'null': 26, 'model': 27, 'brand': 28}


In [34]:
reqs=['cost','color','dualsim','sim','year','month','ram','camera','display','battery','weight','os','rating','gps','month','rec_slt','model'] 
ind_to_span=[('null','null')]
mp_ug={'null':0}
i=1
for row in U_Action:
  for a in row:
    if a not in mp_ug and a in reqs:
      mp_ug[a]=i 
      ind_to_span.append(('request',a))
      i+=1
# targets = np.array([request_span_ind]).reshape(-1)
# one_hot_targets = np.eye(i)[targets]
print(i)
mp_ug_ug={} 
for k in mp_ug:
  mp_ug_ug[k]=[] 
  for ind in range(i):
    if mp_ug[k]==ind:
      mp_ug_ug[k].append(1) 
    else:
      mp_ug_ug[k].append(0) 
request_arr=[] 
Request=[]
for row in U_Action:
  Request.append([])
  for a in row:
    if a in reqs:
        request_arr.append(mp_ug_ug[a]) 
        Request[-1].append(mp_ug_ug[a])
    else:
        Request[-1].append(mp_ug_ug['null'])
        request_arr.append(mp_ug_ug['null'])
print(len(request_arr))
print((ind_to_span[np.argmax(request_arr[1])])) 

17
8384
('request', 'rec_slt')


In [35]:
first_arr=[] 
first_emb=[] 
first_mp=[]
first=[]
cnt=[]
for k in range(15):
  first_arr.append([]) 
  first_emb.append([]) 
  first_mp.append({}) 
  first.append([])
  cnt.append(0) 
  for res in agent_arr:
    if res[k] not in first_mp[k]:
      first_mp[k][res[k]]=cnt[k] 
      first[k].append(res[k])
      cnt[k]+=1
  print(cnt[k]) 
  for i in range(len(User_Arr)):
    first_arr[k].append([])
    if agent_arr[i][k] in word_to_emb:
      first_emb[k].append(word_to_emb[agent_arr[i][k]]) 
    else:
      first_emb[k].append(word_to_emb['place'])
    for j in range(cnt[k]):
      if first_mp[k][agent_arr[i][k]]==j:
        first_arr[k][i].append(1)
      else:
        first_arr[k][i].append(0) 
  print(first_arr[k][0])

356
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [36]:
org_agent_arr=[] 
for i in range(len(User_Arr)):
  org_agent_arr.append(padd(org_Agent_Arr[i]))
inp=[]
for k in range(15): 
  inp.append([])
  for i in range(len(User_Arr)):
    inp[k].append([])
    if i>=4402 and i<4502:
        inp[k][i].append([1]*50) 
    else:
        inp[k][i].append([0]*50)  
    for j in range(15):
      if user_arr[i][j] in word_to_emb:
        inp[k][i].append(word_to_emb[user_arr[i][j]]) 
      else:
        inp[k][i].append(word_to_emb['<unk>'])
print(len(inp[0][0]))
for k in range(1,15):
  for i in range(len(User_Arr)):
    for j in range(k):
      if agent_arr[i][j] in word_to_emb:
        inp[k][i].append(word_to_emb[agent_arr[i][j]])
      elif agent_arr[i][j] in clean_mp:
        inp[k][i].append(word_to_emb[clean_mp[agent_arr[i][j]]])
      else:
        inp[k][i].append(word_to_emb['place'])

16


In [37]:
begin_key={}
curr=0
for i in Agent:
    begin_key[curr]=1 
    curr+=len(i)

In [38]:
inp_sent=[]
temp_sent=['model_name', 'by', 'samsung', 'will','be', 'good', 'for', 'you','.','<pad>','<pad>','<pad>','<pad>','<pad>','<pad>']
for k in range(15): 
  inp_sent.append([])
  for i in range(len(User_Arr)):
    inp_sent[k].append([])
    if i>=4402 and i<4502:
        inp_sent[k][i].append([1]*50) 
        for j in range(15):
            if temp_sent[j] in word_to_emb:
                inp_sent[k][i].append(word_to_emb[temp_sent[j]]) 
            else:
                inp_sent[k][i].append(word_to_emb['<unk>'])
    else:
        inp_sent[k][i].append([0]*50) 
        for j in range(15):
            if i not in begin_key and agent_arr[i-1][j] in word_to_emb :
                inp_sent[k][i].append(word_to_emb[agent_arr[i-1][j]]) 
            else:
                inp_sent[k][i].append(word_to_emb['<unk>']) 
    for j in range(15):
      if user_arr[i][j] in word_to_emb:
        inp_sent[k][i].append(word_to_emb[user_arr[i][j]]) 
      else:
        inp_sent[k][i].append(word_to_emb['<unk>'])
print(len(inp_sent[0][0]))
for k in range(1,15):
  for i in range(len(User_Arr)):
    for j in range(k):
      if agent_arr[i][j] in word_to_emb:
        inp_sent[k][i].append(word_to_emb[agent_arr[i][j]])
      elif agent_arr[i][j] in clean_mp:
        inp_sent[k][i].append(word_to_emb[clean_mp[agent_arr[i][j]]])
      else:
        inp_sent[k][i].append(word_to_emb['place'])

31


In [39]:
value_on_slot={} 
values_of_slot={}
for i in range(len(slot)):
  value_on_slot[slot[i]]={} 
  values_of_slot[slot[i]]=[]
for i in range(len(User_Arr)):
  for j in range(len(Slot_Arr[i])):
    if Value_Arr[i][j] not in values_of_slot[Slot_Arr[i][j]]:
      values_of_slot[Slot_Arr[i][j]].append(Value_Arr[i][j]) 
for k,v in values_of_slot.items():
  for value in v:
    value_on_slot[k][value]=[0]*len(v) 
    value_on_slot[k][value][v.index(value)]=1 


In [40]:
inp_slot=[] 
out_slot=[] 
for i in range(len(User_Arr)):
  for j in range(len(Slot_Arr[i])):
    out_slot.append([])
    inp_slot.append([]) 
    for k in range(15):
      inp_slot[-1].append(user_emb[i][k]) 
    for k in range(len(slot_arr[i])):
      if mp_sl[Slot_Arr[i][j]]==k:
        out_slot[-1].append(1) 
      else:
        out_slot[-1].append(0) 

In [41]:
inp_val=[] 
out_val=[]
for i in range(len(User_Arr)):
  for j in range(len(Value_Arr[i])):
    out_val.append([])
    inp_val.append([]) 
    for k in range(15):
      inp_val[-1].append(user_emb[i][k]) 
    inp_val[-1].append(word_to_emb[slot[np.argmax(out_slot[len(inp_val)-1])]]) 
    for k in range(len(value)):
      if mp_vl[Value_Arr[i][j]]==k:
        out_val[-1].append(1) 
      else:
        out_val[-1].append(0) 
# import random
# def not_present_fun(bspan):
#     f=open('dialogue.txt',encoding='utf-8')
#     arr=[] 
#     for line in f:
#         arr.append(line)
#     index_of_sent=random.randint(0,len(arr)-1)
#     return arr[index_of_sent]

In [42]:
copy_arr=[]
copy_slot={}
copy_value={}
for i in range(25):
  copy_slot[slot[i]]=[] 
  copy_value[slot[i]]=[] 
for i in range(len(User_Arr)):
  words=word_tokenize(User_Arr[i])
  for j in range(len(Slot_Arr[i])):
    for word in words:
      if word in word_to_emb:
        copy_slot[Slot_Arr[i][j]].append(word_to_emb[word]) 
      else:
        copy_slot[Slot_Arr[i][j]].append(word_to_emb['blank'])
      if Value_Arr[i][j]==word:
        copy_value[Slot_Arr[i][j]].append([0,1])
        if Slot_Arr[i][j]=='color':
            print(word)
      else:
        copy_value[Slot_Arr[i][j]].append([1,0]) 

black
white
grey
white
gold
gold
black
blue
silver
blue
black
blue
gold
black
pink
gold
white
grey
gold
black
silver
grey
gold
white
gold
black
gold
white
white
black
black
green
white
silver
black
black
green
black
black
white
grey
gold
white
blue
white
black
white
gold
chrome
gray
gold
silver
grey
gold
white
gold
blue
blue
black
blue
gold
gold
black
black
silver
pink
black
orange
black
black
black
white
grey
black
black
black
black
black
black
gold
black
blue
gold
black
blue
white
grey
grey
blue
black
blue
black
blue
black
black
black
blue
blue
gray
black
blue
blue
black
gold
gold
silver
black
blue
yellow
gold
white
black
gold
black
black
black
white
silver
green
black
blue
white
gold
titanium
black
black
blue
pink
blue
pink
black
silver
pink
black
blue
black
black
metal
black
blue
blue
black
silver
black
good
silver
pink
blue
yellow
black
gold
silver
black
gold
silver
white
silver
pink
grey
black
blue
blue
gray
gold
silver
black
blue
green
white
white
black
black
black
orange
black


In [43]:
for k in value_on_slot:
  for v in value_on_slot[k]:
    if len(value_on_slot[k][v])==1:
      value_on_slot[k][v]=[1,0]
value_slot={}
value_value={}
for i in range(25):
  value_slot[slot[i]]=[] 
  value_value[slot[i]]=[] 
for i in range(len(User_Arr)):
  words=word_tokenize(User_Arr[i])
  for j in range(len(Slot_Arr[i])):
    value_slot[Slot_Arr[i][j]].append(user_emb[i]) 
    value_value[Slot_Arr[i][j]].append(value_on_slot[Slot_Arr[i][j]][Value_Arr[i][j]]) 
value_value['null'][0]

[1, 0]

In [44]:
def isstr(word):
  try:
    word=int(word) 
    word=float(word) 
  except:
    word=word 
  if type(word)==str:
    return 1 
  else:
    return 0
temp={}
for i in range(len(User_Arr)):
  if U_Action_Arr[i] not in temp:
    temp[U_Action_Arr[i]]=i 
    #print(U_Action_Arr[i],Agent_Arr[i])
print(temp) 
temp=[]
for i in range(len(User_Arr)):
  for word in agent_arr[i]:
    if word not in word_to_id and word not in brand_arr and word!='ref_no':
      if isstr(word)==1:
        temp.append(word) 
tmp_mp={} 
for t in temp:
  if 'gms' in t:
    tmp_mp[t]=1 
print(tmp_mp)
print(temp)

{'phone': 0, 'rec_slt': 1, 'cost': 2, 'color': 3, 'feedback': 4, 'booking': 7, 'thanks': 8, 'dualsim': 21, 'year': 22, 'os': 23, 'camera': 31, 'rating': 32, 'confirm': 33, 'battery': 41, 'memory': 53, 'review': 74, 'specification': 95, 'discount': 99, 'weight': 244, 'display': 253, 'radio': 269, 'ram': 276, 'greet': 302, 'month': 372, 'sim': 730, 'gps': 950, 'null': 2333, 'model': 4210, 'brand': 4413}
{'model_name70gms': 1, '137gms': 1}
['model_name', 'brand_name', 'money_unit', 'cost_price', 'color_name', 'model_nameas', 'color_name', 'model_name', 'color_namemi', 'cost_price', 'model_name', 'brand_name', 'money_unit', 'cost_price', 'color_name', 'color_name', 'color_name', 'model_namey', 'brand_name', 'money_unit', 'cost_price', 'model_name', 'money_unit', 'cost_price', 'year_name', 'os_name', 'model_name', 'brand_name', 'money_unit', 'cost_price', 'model_name', 'brand_name', 'cam_val', 'rating_score', 'cost_price', 'model_name', 'brand_name', 'color_name', 'model_name', 'color_namem

In [45]:
def arg3max(arr):
  temp_arr=[] 
  for i in range(len(arr)):
    temp_arr.append((arr[i],i)) 
  temp_arr.sort() 
  temp_arr.reverse() 
  ans_arr=[] 
  for i in range(3):
    ans_arr.append((temp_arr[i][0],temp_arr[i][1],0)) 
  return ans_arr 
def combineArg3max(arr,p):
  temp_arr=[] 
  for i in range(len(arr[0])):
    temp_arr.append((100*p[0]*arr[0][i],i,0))  
    temp_arr.append((100*p[1]*arr[1][i],i,1))
    temp_arr.append((100*p[2]*arr[2][i],i,2))
  temp_arr.sort() 
  temp_arr.reverse() 
  ans_arr=[] 
  for i in range(3):
    ans_arr.append(temp_arr[i])   
  return ans_arr
def reward(word,bspan):
    if bspan[1]=='null':
        return 0
    for k,v in slot_to_key.items():
        if v==word and k==bspan[1]:
            return 1 
        elif v==word and k!=bspan[1]:
            #print(word,k,bspan[1])
            return -1
    return -0.05
def beam_search(inp_arr,model,first,bspan,is_present,pers):
  inp_arr_temp=[] 
  for i in range(16):
    inp_arr_temp.append(inp_arr[i]) 
  dial_arr=[[],[],[]] 
  p=[]
  out0=model[0].predict([np.array([inp_arr]),np.array([inp_arr])]) 
  temp=arg3max(out0[0])
  temp_dial=[]
  for i in range(3): 
    dial_arr[i].append(first[0][temp[i][1]])  
  for i in range(3):
    reward_val=reward(dial_arr[i][0],bspan)
    if reward_val==-1 and slot_to_key[bspan[1]] not in dial_arr[i]:
        dial_arr[i][0]=slot_to_key[bspan[1]]
    p.append(temp[i][0]+reward_val) 
  for j in range(1,15):
    #print(dial_arr)
    temp_arr=[]
    for i in range(3):
      for k in range(len(dial_arr[i])):
        #print(dial_arr[i][k])
        if dial_arr[i][k] in word_to_emb:
          inp_arr_temp.append(word_to_emb[dial_arr[i][k]]) 
        else:
          inp_arr_temp.append(word_to_emb['place'])
      temp_arr.append(model[j].predict([np.array([inp_arr_temp]),np.array([inp_arr_temp])])[0]) 
      for k in range(len(dial_arr[i])):
        inp_arr_temp.pop() 
    temp=combineArg3max(temp_arr,p) 
    temp_dial=[]
    for i in range(3): 
      dial_arr[temp[i][2]].append(first[j][temp[i][1]])
      temp_dial.append([]) 
      for k in range(len(dial_arr[temp[i][2]])):
        temp_dial[i].append(dial_arr[temp[i][2]][k]) 
      dial_arr[temp[i][2]].pop()   
    for i in range(3):
      dial_arr[i]=temp_dial[i]
    for i in range(3):
      reward_val=reward(dial_arr[i][j],bspan)
      if reward_val==-1 and slot_to_key[bspan[1]] not in dial_arr[i]:
            dial_arr[i][j]=slot_to_key[bspan[1]]
      p[i]=temp[i][0]+reward_val  
      if dial_arr[0][0]=='brand_name':
            dial_arr[0][0]='model_name'
  ret=' '.join(dial_arr[0]).replace('<pad>','') 
  if pers==-1:
        ret.replace('brand_name','it')
        ret.replace('euro','has')
        ret+="it has best_feature."
  return ret
def whole_beam_search(inp_arr,model,first):
  output=[]
  for i in range(len(inp_arr)):
    output.append(beam_search(inp_arr[i],model,first)) 
  return output 


In [46]:
inp_encoder=[]
mx_encoder_len=0 
pos=0
for i in range(1000):
  for j in range(len(User[i])):
    inp_encoder.append([])
    if j==0:
      bspans=[]
      for k in range(15):
        inp_encoder[-1].append([0]*50) 
    else:
      #bspans adding 
      for sl in range(len(Slot_Arr[pos-1])):
        if Slot_Arr[pos-1][sl]!='null':
            bspans.append((Slot_Arr[pos-1][sl],Value_Arr[pos-1][sl]))
      sent=padd(Agent[i][j-1])
      for k in range(15):
        if sent[k] in word_to_emb:
          inp_encoder[-1].append(word_to_emb[sent[k]])
        else:
          inp_encoder[-1].append(word_to_emb['<unk>']) 
    
    sent=padd(User[i][j]) 
    for k in range(15):
      if sent[k] in word_to_emb:
        inp_encoder[-1].append(word_to_emb[sent[k]]) 
      else:
        inp_encoder[-1].append(word_to_emb['place']) 
    for k in range(len(bspans)):
      for l in range(2):
        if bspans[k][l] in word_to_emb:
          inp_encoder[-1].append(word_to_emb[bspans[k][l]]) 
        else:
          inp_encoder[-1].append(word_to_emb['place'])  
    mx_encoder_len=max(len(inp_encoder[-1]),mx_encoder_len)
    pos+=1
print(pos)
for i in range(len(inp_encoder)):
  inp_len=len(inp_encoder[i])
  for j in range(36-inp_len):
    inp_encoder[i].append(word_to_emb['<unk>']) 
  inp_encoder[i]=inp_encoder[i][:36]

8384


In [47]:
print(len(inp_encoder))
for i in range(len(User_Arr)):
    if len(inp_encoder[i])!=36:
        print(i,len(inp_encoder[i]))
        print(Slot_Arr[i])

8384


In [48]:
def update_bspan(bspan_arr,d_index):
    for k in bspan_arr:
        bspan_arr[k]=db[d_index][slot_to_key[k]]
    return bspan_arr

In [65]:
def seq2seq(inp_encoder_np,out_encoder_np,decoder_output_np,out_len):
    # Define an input sequence and process it.
    encoder_inputs = Input(shape=(None, 50))
    encoder = LSTM(50, return_state=True) 
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    #l_encoder = list(encoder(encoder_inputs))
    #print(len(l_encoder),l_encoder[0].shape)
    # We discard `encoder_outputs` and only keep the states.
    encoder_states = [state_h, state_c]
    print(state_h.shape)
    enc_dense=Dense(50,activation='relu') 
    enc_out=enc_dense(encoder_outputs)
    encoder_model = Model(encoder_inputs,encoder_outputs) 
    # Set up the decoder, using `encoder_states` as initial state.
    decoder_inputs = Input(shape=(None, 50))
    # We set up our decoder to return full output sequences,
    # and to return internal states as well. We don't use the 
    # return states in the training model, but we will use them in inference.
    decoder_lstm = LSTM(50,return_state=True)
    decoder_outputs, s1, s2 = decoder_lstm(decoder_inputs,
                                         initial_state=encoder_states)
    decoder_state=[s1,s2]
    decoder_dense = Dense(out_len, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
    # Define the model that will turn
    # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])
    model.fit([inp_encoder_np, out_encoder_np], decoder_output_np,
              epochs=20)
    print(model.summary())
    return model,encoder_model,encoder_states
def decoderModel(inp_decoder_state,inp_decoder_np,out_decoder_np,out_len):
#     inp_decoder_h=Input(shape=(None,50)) 
#     inp_decoder_c=Input(shape=(None,50))
#     inp_state=inp_decoder_state
    decoder_inputs = Input(shape=(12,50)) 
    # We set up our decoder to return full output sequences,
    # and to return internal states as well. We don't use the 
    # return states in the training model, but we will use them in inference.
    decoder_lstm = LSTM(50, return_state=True)
    decoder_outputs, s1, s2 = decoder_lstm(decoder_inputs)
    decoder_state=[s1,s2]
    decoder_dense = Dense(out_len, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs) 
    decoder_model=Model(decoder_inputs,decoder_outputs) 
    decoder_model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=['accuracy'])
    decoder_model.fit(inp_encoder_np, out_decoder_np,
              epochs=1,
              validation_split=0.2)
    return decoder_model

In [50]:
def model1(X,Y,input_size,output_size):
  m=Sequential() 
  m.add(Bidirectional(LSTM(256,input_shape=(None,input_size)))) 
  m.add(Dense(32,activation='relu'))
  #m.add(Dropout(0.5))
  m.add(Dense(output_size,activation='softmax')) 
  m.compile('Adam','categorical_crossentropy',['accuracy'])
  m.fit(X,Y,epochs=15)
  return m 

In [51]:
def model2(X,Y,input_size,output_size):
  m=Sequential() 
  #m.add(Bidirectional(LSTM(256,input_shape=(1,input_size))))
  #m.add(Dropout(0.5))
  #m.add(Dense(56,activation='relu')) 
  m.add(Dense(32,activation='relu'))
  #m.add(Dropout(0.5))
  m.add(Dense(output_size,activation='softmax')) 
  m.compile('Adam','categorical_crossentropy',['accuracy'])
  m.fit(X,Y,epochs=12)
  return m 

In [52]:
sentiment_model=model1(np.array(sentiment_inp),np.array(sentiment_label),15,3)

Epoch 1/15
252/252 [==============================] - 11s 6ms/step - loss: 0.4840 - accuracy: 0.8152
Epoch 2/15
252/252 [==============================] - 2s 6ms/step - loss: 0.2264 - accuracy: 0.9200
Epoch 3/15
252/252 [==============================] - 2s 6ms/step - loss: 0.1883 - accuracy: 0.9311
Epoch 4/15
252/252 [==============================] - 2s 6ms/step - loss: 0.1587 - accuracy: 0.9494
Epoch 5/15
252/252 [==============================] - 2s 6ms/step - loss: 0.1272 - accuracy: 0.9573
Epoch 6/15
252/252 [==============================] - 1s 6ms/step - loss: 0.1161 - accuracy: 0.9592
Epoch 7/15
252/252 [==============================] - 1s 6ms/step - loss: 0.1029 - accuracy: 0.9649
Epoch 8/15
252/252 [==============================] - 2s 6ms/step - loss: 0.0848 - accuracy: 0.9711
Epoch 9/15
252/252 [==============================] - 2s 6ms/step - loss: 0.0882 - accuracy: 0.9688
Epoch 10/15
252/252 [==============================] - 2s 6ms/step - loss: 0.0709 - accuracy: 0.977

In [53]:
print(np.array(request_arr).shape)
request_model,_,_=seq2seq(np.array(user_emb[:7095]),np.array(user_emb[:7095]),np.array(request_arr[:7095]),17)
request_model.save('saved_models/request_model')
request_model=keras.models.load_model('saved_models/request_model')

(8384, 17)
(None, 50)
Epoch 1/15
222/222 [==============================] - 4s 6ms/step - loss: 1.3476 - accuracy: 0.6374
Epoch 2/15
222/222 [==============================] - 1s 6ms/step - loss: 0.5166 - accuracy: 0.8482
Epoch 3/15
222/222 [==============================] - 1s 6ms/step - loss: 0.3145 - accuracy: 0.9147
Epoch 4/15
222/222 [==============================] - 1s 6ms/step - loss: 0.2479 - accuracy: 0.9298
Epoch 5/15
222/222 [==============================] - 1s 6ms/step - loss: 0.2084 - accuracy: 0.9392
Epoch 6/15
222/222 [==============================] - 1s 6ms/step - loss: 0.1695 - accuracy: 0.9526
Epoch 7/15
222/222 [==============================] - 1s 6ms/step - loss: 0.1662 - accuracy: 0.9500
Epoch 8/15
222/222 [==============================] - 1s 6ms/step - loss: 0.1435 - accuracy: 0.9581
Epoch 9/15
222/222 [==============================] - 1s 6ms/step - loss: 0.1380 - accuracy: 0.9613
Epoch 10/15
222/222 [==============================] - 1s 6ms/step - loss: 0.1

INFO:tensorflow:Assets written to: saved_models/request_model/assets


INFO:tensorflow:Assets written to: saved_models/request_model/assets


In [54]:
request_model.evaluate([np.array(user_emb[7095:]),np.array(user_emb[7095:])],np.array(request_arr[7095:]))

41/41 [==============================] - 1s 4ms/step - loss: 0.1185 - accuracy: 0.9682


[0.1185179278254509, 0.9681923985481262]

In [55]:
tp,fn,fp=0,0,0 
for i in range(7095,len(inp_encoder)):
    emb_dial=sent_to_emb(User_Arr[i])
    request_span=request_model.predict([np.array([emb_dial]),np.array([emb_dial])])[0]
    gen_span=ind_to_span[np.argmax(request_span)][1]
    true_span=ind_to_span[np.argmax(request_arr[i])][1]
    if true_span not in ['rec_slt','null']:
        #print(true_span,gen_span)
        if true_span==gen_span:
            tp+=1 
        else:
            fn+=1
    if gen_span not in ['rec_slt','null'] and gen_span!=true_span:
        fp+=1
precision=tp/(tp+fp) 
recall=tp/(tp+fn)
print("precision=",precision)
print("recall=",recall)
successf1=(precision*recall*2)/(precision+recall)
print("successf1=",successf1)

precision= 0.9735973597359736
recall= 0.9768211920529801
successf1= 0.975206611570248


In [56]:
copy_model={} 
for i in range(25):
  copy_model[slot[i]]=model2(np.array(copy_slot[slot[i]]),np.array(copy_value[slot[i]]),50,2)
  copy_model[slot[i]].save('saved_models/copy_model'+str(i))
for i in range(25):
    copy_model[slot[i]]=keras.models.load_model('saved_models/copy_model'+str(i))

Epoch 1/12
1344/1344 [==============================] - 3s 2ms/step - loss: 0.1941 - accuracy: 0.9003
Epoch 2/12
1344/1344 [==============================] - 3s 2ms/step - loss: 2.1568e-04 - accuracy: 1.0000
Epoch 3/12
1344/1344 [==============================] - 3s 2ms/step - loss: 4.9326e-05 - accuracy: 1.0000
Epoch 4/12
1344/1344 [==============================] - 3s 2ms/step - loss: 1.6375e-05 - accuracy: 1.0000
Epoch 5/12
1344/1344 [==============================] - 3s 2ms/step - loss: 7.2999e-06 - accuracy: 1.0000
Epoch 6/12
1344/1344 [==============================] - 3s 2ms/step - loss: 3.0170e-06 - accuracy: 1.0000
Epoch 7/12
1344/1344 [==============================] - 3s 2ms/step - loss: 1.3038e-06 - accuracy: 1.0000
Epoch 8/12
1344/1344 [==============================] - 3s 2ms/step - loss: 7.2564e-07 - accuracy: 1.0000
Epoch 9/12
1344/1344 [==============================] - 3s 2ms/step - loss: 3.0418e-07 - accuracy: 1.0000
Epoch 10/12
1344/1344 [===========================

INFO:tensorflow:Assets written to: saved_models/copy_model0/assets


Epoch 1/12
143/143 [==============================] - 1s 2ms/step - loss: 0.2602 - accuracy: 0.9181
Epoch 2/12
143/143 [==============================] - 0s 2ms/step - loss: 0.0709 - accuracy: 0.9851
Epoch 3/12
143/143 [==============================] - 0s 2ms/step - loss: 0.0288 - accuracy: 0.9984
Epoch 4/12
143/143 [==============================] - 0s 2ms/step - loss: 0.0193 - accuracy: 0.9973
Epoch 5/12
143/143 [==============================] - 0s 2ms/step - loss: 0.0156 - accuracy: 0.9967
Epoch 6/12
143/143 [==============================] - 0s 2ms/step - loss: 0.0200 - accuracy: 0.9932
Epoch 7/12
143/143 [==============================] - 0s 2ms/step - loss: 0.0178 - accuracy: 0.9951
Epoch 8/12
143/143 [==============================] - 0s 2ms/step - loss: 0.0130 - accuracy: 0.9969
Epoch 9/12
143/143 [==============================] - 0s 2ms/step - loss: 0.0134 - accuracy: 0.9971
Epoch 10/12
143/143 [==============================] - 0s 2ms/step - loss: 0.0113 - accuracy: 0.9968

INFO:tensorflow:Assets written to: saved_models/copy_model1/assets


Epoch 1/12
87/87 [==============================] - 0s 2ms/step - loss: 0.5367 - accuracy: 0.7099
Epoch 2/12
87/87 [==============================] - 0s 2ms/step - loss: 0.1232 - accuracy: 0.9507
Epoch 3/12
87/87 [==============================] - 0s 2ms/step - loss: 0.0513 - accuracy: 0.9946
Epoch 4/12
87/87 [==============================] - 0s 2ms/step - loss: 0.0364 - accuracy: 0.9951
Epoch 5/12
87/87 [==============================] - 0s 2ms/step - loss: 0.0278 - accuracy: 0.9956
Epoch 6/12
87/87 [==============================] - 0s 2ms/step - loss: 0.0317 - accuracy: 0.9923
Epoch 7/12
87/87 [==============================] - 0s 2ms/step - loss: 0.0187 - accuracy: 0.9963
Epoch 8/12
87/87 [==============================] - 0s 2ms/step - loss: 0.0232 - accuracy: 0.9941
Epoch 9/12
87/87 [==============================] - 0s 2ms/step - loss: 0.0168 - accuracy: 0.9962
Epoch 10/12
87/87 [==============================] - 0s 2ms/step - loss: 0.0114 - accuracy: 0.9979
Epoch 11/12
87/87 [

...

INFO:tensorflow:Assets written to: saved_models/copy_model22/assets


Epoch 1/12
1/1 [==============================] - 0s 292ms/step - loss: 0.4279 - accuracy: 0.8333
Epoch 2/12
1/1 [==============================] - 0s 4ms/step - loss: 0.4010 - accuracy: 0.8333
Epoch 3/12
1/1 [==============================] - 0s 3ms/step - loss: 0.3753 - accuracy: 0.8333
Epoch 4/12
1/1 [==============================] - 0s 9ms/step - loss: 0.3510 - accuracy: 0.8333
Epoch 5/12
1/1 [==============================] - 0s 3ms/step - loss: 0.3283 - accuracy: 0.8333
Epoch 6/12
1/1 [==============================] - 0s 5ms/step - loss: 0.3072 - accuracy: 0.9167
Epoch 7/12
1/1 [==============================] - 0s 4ms/step - loss: 0.2873 - accuracy: 0.9167
Epoch 8/12
1/1 [==============================] - 0s 4ms/step - loss: 0.2686 - accuracy: 0.9167
Epoch 9/12
1/1 [==============================] - 0s 4ms/step - loss: 0.2511 - accuracy: 0.9167
Epoch 10/12
1/1 [==============================] - 0s 4ms/step - loss: 0.2347 - accuracy: 0.9167
Epoch 11/12
1/1 [====================

INFO:tensorflow:Assets written to: saved_models/copy_model23/assets


Epoch 1/12
2/2 [==============================] - 0s 6ms/step - loss: 1.5791 - accuracy: 0.1100
Epoch 2/12
2/2 [==============================] - 0s 5ms/step - loss: 1.4810 - accuracy: 0.1100
Epoch 3/12
2/2 [==============================] - 0s 4ms/step - loss: 1.3952 - accuracy: 0.1100
Epoch 4/12
2/2 [==============================] - 0s 4ms/step - loss: 1.3058 - accuracy: 0.1100
Epoch 5/12
2/2 [==============================] - 0s 10ms/step - loss: 1.2166 - accuracy: 0.1100
Epoch 6/12
2/2 [==============================] - 0s 5ms/step - loss: 1.1515 - accuracy: 0.0996
Epoch 7/12
2/2 [==============================] - 0s 4ms/step - loss: 1.0796 - accuracy: 0.0996
Epoch 8/12
2/2 [==============================] - 0s 4ms/step - loss: 1.0177 - accuracy: 0.0996
Epoch 9/12
2/2 [==============================] - 0s 3ms/step - loss: 0.9456 - accuracy: 0.1651
Epoch 10/12
2/2 [==============================] - 0s 6ms/step - loss: 0.8962 - accuracy: 0.1613
Epoch 11/12
2/2 [=====================

INFO:tensorflow:Assets written to: saved_models/copy_model24/assets


In [57]:
m1=[] 
for i in range(25):
  m1.append(seq2seq(np.array(value_slot[slot[i]]),np.array(value_slot[slot[i]]),np.array(value_value[slot[i]]),len(value_value[slot[i]][0]))[0])
  m1[i].save('saved_models/value_model'+str(i))
# for i in range(25):
#     m1.append(keras.models.load_model('saved_models/value_model'+str(i)))

(None, 50)
Epoch 1/15
190/190 [==============================] - 4s 6ms/step - loss: 0.0659 - accuracy: 0.9914
Epoch 2/15
190/190 [==============================] - 1s 6ms/step - loss: 2.4420e-07 - accuracy: 1.0000
Epoch 3/15
190/190 [==============================] - 1s 6ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/15
190/190 [==============================] - 1s 6ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/15
190/190 [==============================] - 1s 6ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/15
190/190 [==============================] - 1s 6ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/15
190/190 [==============================] - 1s 6ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/15
190/190 [==============================] - 1s 6ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/15
190/190 [==============================] - 1s 6ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/15
190/190 [==============================] - 1s

INFO:tensorflow:Assets written to: saved_models/value_model0/assets


INFO:tensorflow:Assets written to: saved_models/value_model0/assets


(None, 50)
Epoch 1/15
13/13 [==============================] - 2s 6ms/step - loss: 2.2166 - accuracy: 0.5784
Epoch 2/15
13/13 [==============================] - 0s 6ms/step - loss: 1.2601 - accuracy: 0.7272
Epoch 3/15
13/13 [==============================] - 0s 6ms/step - loss: 1.1503 - accuracy: 0.7256
Epoch 4/15
13/13 [==============================] - 0s 6ms/step - loss: 1.0075 - accuracy: 0.7310
Epoch 5/15
13/13 [==============================] - 0s 6ms/step - loss: 0.9173 - accuracy: 0.7454
Epoch 6/15
13/13 [==============================] - 0s 6ms/step - loss: 0.7840 - accuracy: 0.7746
Epoch 7/15
13/13 [==============================] - 0s 7ms/step - loss: 0.7000 - accuracy: 0.7977
Epoch 8/15
13/13 [==============================] - 0s 6ms/step - loss: 0.7089 - accuracy: 0.8082
Epoch 9/15
13/13 [==============================] - 0s 6ms/step - loss: 0.6949 - accuracy: 0.8079
Epoch 10/15
13/13 [==============================] - 0s 6ms/step - loss: 0.6456 - accuracy: 0.8251
Epoch 11

INFO:tensorflow:Assets written to: saved_models/value_model1/assets


...

INFO:tensorflow:Assets written to: saved_models/value_model23/assets


INFO:tensorflow:Assets written to: saved_models/value_model23/assets


(None, 50)
Epoch 1/15
1/1 [==============================] - 2s 2s/step - loss: 0.7906 - accuracy: 0.0000e+00
Epoch 2/15
1/1 [==============================] - 0s 9ms/step - loss: 0.5548 - accuracy: 1.0000
Epoch 3/15
1/1 [==============================] - 0s 8ms/step - loss: 0.3987 - accuracy: 1.0000
Epoch 4/15
1/1 [==============================] - 0s 9ms/step - loss: 0.2749 - accuracy: 1.0000
Epoch 5/15
1/1 [==============================] - 0s 10ms/step - loss: 0.1853 - accuracy: 1.0000
Epoch 6/15
1/1 [==============================] - 0s 10ms/step - loss: 0.1262 - accuracy: 1.0000
Epoch 7/15
1/1 [==============================] - 0s 9ms/step - loss: 0.0890 - accuracy: 1.0000
Epoch 8/15
1/1 [==============================] - 0s 9ms/step - loss: 0.0654 - accuracy: 1.0000
Epoch 9/15
1/1 [==============================] - 0s 9ms/step - loss: 0.0499 - accuracy: 1.0000
Epoch 10/15
1/1 [==============================] - 0s 9ms/step - loss: 0.0392 - accuracy: 1.0000
Epoch 11/15
1/1 [======

INFO:tensorflow:Assets written to: saved_models/value_model24/assets


INFO:tensorflow:Assets written to: saved_models/value_model24/assets


In [58]:
m2=seq2seq(np.array(inp_slot[:7798]),np.array(inp_slot[:7798]),np.array(out_slot[:7798]),25)[0]
m2.save('saved_models/slot_model')
m2=keras.models.load_model('saved_models/slot_model')

(None, 50)
Epoch 1/15
244/244 [==============================] - 4s 6ms/step - loss: 1.5875 - accuracy: 0.6599
Epoch 2/15
244/244 [==============================] - 2s 6ms/step - loss: 0.9588 - accuracy: 0.7225
Epoch 3/15
244/244 [==============================] - 2s 6ms/step - loss: 0.7296 - accuracy: 0.7852
Epoch 4/15
244/244 [==============================] - 2s 6ms/step - loss: 0.6388 - accuracy: 0.8046
Epoch 5/15
244/244 [==============================] - 2s 6ms/step - loss: 0.5456 - accuracy: 0.8323
Epoch 6/15
244/244 [==============================] - 2s 6ms/step - loss: 0.4973 - accuracy: 0.8380
Epoch 7/15
244/244 [==============================] - 1s 6ms/step - loss: 0.4532 - accuracy: 0.8436
Epoch 8/15
244/244 [==============================] - 2s 6ms/step - loss: 0.4150 - accuracy: 0.8582
Epoch 9/15
244/244 [==============================] - 2s 6ms/step - loss: 0.4179 - accuracy: 0.8531
Epoch 10/15
244/244 [==============================] - 2s 6ms/step - loss: 0.3741 - accur

INFO:tensorflow:Assets written to: saved_models/slot_model/assets


INFO:tensorflow:Assets written to: saved_models/slot_model/assets


In [59]:
m2.evaluate([np.array(user_emb[7095:]),np.array(user_emb[7095:])],np.array(slot_arr[7095:]))

41/41 [==============================] - 1s 3ms/step - loss: 0.4226 - accuracy: 0.8976


[0.42263051867485046, 0.8975950479507446]

In [60]:
out_slot_pred=m2.predict([np.array(user_emb[7095:len(User_Arr)]),np.array(user_emb[7095:len(User_Arr)])])
pre_tot,rec_tot,tp=0,0,0
for i in range(7095,len(User_Arr)):
    x,y=get_pos(out_slot_pred[i-7095]),get_pos(slot_arr[i])
    for v in x:
        if v in y:
            tp+=1
    pre_tot+=len(x) 
    rec_tot+=len(y)
precision=tp/pre_tot 
recall=tp/rec_tot
print('precision=',precision) 
print('recall=',recall) 

precision= 0.8638676844783715
recall= 0.9590395480225988


In [61]:
print('f-measure=',(precision*recall*2)/(precision+recall))

f-measure= 0.9089692101740294


In [62]:
# sent_m=[]
# for i in range(15):  
#   sent_m.append(seq2seq(np.array(inp_sent[i][:6995]),np.array(inp_sent[i][:6995]),np.array(first_arr[i][:6995]),cnt[i])[0]) 
#   sent_m[i].save('saved_models/sent_m'+str(i))
# for i in range(12):
#     sent_m.append(keras.models.load_model('saved_models/m'+str(i)))

(None, 50)
Epoch 1/15
219/219 [==============================] - 4s 7ms/step - loss: 4.1199 - accuracy: 0.2244
Epoch 2/15
219/219 [==============================] - 2s 7ms/step - loss: 3.0634 - accuracy: 0.3241
Epoch 3/15
219/219 [==============================] - 2s 7ms/step - loss: 2.7108 - accuracy: 0.4100
Epoch 4/15
219/219 [==============================] - 2s 7ms/step - loss: 2.4935 - accuracy: 0.4560
Epoch 5/15
219/219 [==============================] - 2s 7ms/step - loss: 2.3026 - accuracy: 0.4870
Epoch 6/15
219/219 [==============================] - 2s 7ms/step - loss: 2.2164 - accuracy: 0.5064
Epoch 7/15
219/219 [==============================] - 2s 8ms/step - loss: 2.1775 - accuracy: 0.5030
Epoch 8/15
219/219 [==============================] - 2s 7ms/step - loss: 2.1180 - accuracy: 0.5239
Epoch 9/15
219/219 [==============================] - 2s 7ms/step - loss: 2.0488 - accuracy: 0.5325
Epoch 10/15
219/219 [==============================] - 2s 7ms/step - loss: 1.9908 - accur

INFO:tensorflow:Assets written to: saved_models/sent_m0/assets


INFO:tensorflow:Assets written to: saved_models/sent_m0/assets


(None, 50)
Epoch 1/15
219/219 [==============================] - 4s 8ms/step - loss: 4.2194 - accuracy: 0.2239
Epoch 2/15
219/219 [==============================] - 2s 8ms/step - loss: 2.9247 - accuracy: 0.4131
Epoch 3/15
219/219 [==============================] - 2s 8ms/step - loss: 2.4680 - accuracy: 0.4584
Epoch 4/15
219/219 [==============================] - 2s 8ms/step - loss: 2.1832 - accuracy: 0.5133
Epoch 5/15
219/219 [==============================] - 2s 8ms/step - loss: 2.0687 - accuracy: 0.5367
Epoch 6/15
219/219 [==============================] - 2s 8ms/step - loss: 1.9016 - accuracy: 0.5712
Epoch 7/15
219/219 [==============================] - 2s 8ms/step - loss: 1.7549 - accuracy: 0.6102
Epoch 8/15
219/219 [==============================] - 2s 8ms/step - loss: 1.6748 - accuracy: 0.6227
Epoch 9/15
219/219 [==============================] - 2s 7ms/step - loss: 1.5720 - accuracy: 0.6477
Epoch 10/15
219/219 [==============================] - 2s 7ms/step - loss: 1.5207 - accur

INFO:tensorflow:Assets written to: saved_models/sent_m1/assets


...

INFO:tensorflow:Assets written to: saved_models/sent_m13/assets


INFO:tensorflow:Assets written to: saved_models/sent_m13/assets


(None, 50)
Epoch 1/15
219/219 [==============================] - 4s 9ms/step - loss: 2.0751 - accuracy: 0.8108
Epoch 2/15
219/219 [==============================] - 2s 9ms/step - loss: 0.8123 - accuracy: 0.8464
Epoch 3/15
219/219 [==============================] - 2s 9ms/step - loss: 0.7353 - accuracy: 0.8504
Epoch 4/15
219/219 [==============================] - 2s 9ms/step - loss: 0.6844 - accuracy: 0.8629
Epoch 5/15
219/219 [==============================] - 2s 9ms/step - loss: 0.6812 - accuracy: 0.8597
Epoch 6/15
219/219 [==============================] - 2s 9ms/step - loss: 0.6385 - accuracy: 0.8725
Epoch 7/15
219/219 [==============================] - 2s 9ms/step - loss: 0.6160 - accuracy: 0.8731
Epoch 8/15
219/219 [==============================] - 2s 8ms/step - loss: 0.5777 - accuracy: 0.8815
Epoch 9/15
219/219 [==============================] - 2s 8ms/step - loss: 0.5826 - accuracy: 0.8802
Epoch 10/15
219/219 [==============================] - 2s 9ms/step - loss: 0.5274 - accur

INFO:tensorflow:Assets written to: saved_models/sent_m14/assets


INFO:tensorflow:Assets written to: saved_models/sent_m14/assets


In [63]:
# for i in range(15):
#   print(sent_m[i].evaluate([np.array(inp_sent[i][8095:]),np.array(inp_sent[i][8095:])],np.array(first_arr[i][8095:])))

10/10 [==============================] - 1s 5ms/step - loss: 2.2204 - accuracy: 0.6055
[2.2204041481018066, 0.6055363416671753]
10/10 [==============================] - 1s 5ms/step - loss: 1.9130 - accuracy: 0.6436
[1.9129985570907593, 0.6435986161231995]
10/10 [==============================] - 1s 5ms/step - loss: 2.3345 - accuracy: 0.6055
[2.3344805240631104, 0.6055363416671753]
10/10 [==============================] - 1s 5ms/step - loss: 2.1019 - accuracy: 0.6436
[2.101869821548462, 0.6435986161231995]
10/10 [==============================] - 1s 5ms/step - loss: 2.3493 - accuracy: 0.5917
[2.3493235111236572, 0.5916954874992371]
10/10 [==============================] - 1s 5ms/step - loss: 1.8818 - accuracy: 0.6505
[1.8818061351776123, 0.6505190134048462]
10/10 [==============================] - 1s 5ms/step - loss: 1.5705 - accuracy: 0.7163
[1.5704741477966309, 0.7162629961967468]
10/10 [==============================] - 1s 5ms/step - loss: 1.4018 - accuracy: 0.7647
[1.401763916015625

In [66]:
m=[]
for i in range(15):  
  m.append(seq2seq(np.array(inp[i][:6995]),np.array(inp[i][:6995]),np.array(first_arr[i][:6995]),cnt[i])[0]) 
  m[i].save('saved_models/m'+str(i))
# for i in range(12):
#     m.append(keras.models.load_model('saved_models/m'+str(i)))

(None, 50)
Epoch 1/20
219/219 [==============================] - 4s 7ms/step - loss: 4.0822 - accuracy: 0.2449
Epoch 2/20
219/219 [==============================] - 1s 7ms/step - loss: 3.0221 - accuracy: 0.3636
Epoch 3/20
219/219 [==============================] - 1s 6ms/step - loss: 2.6802 - accuracy: 0.4300
Epoch 4/20
219/219 [==============================] - 1s 7ms/step - loss: 2.5057 - accuracy: 0.4498
Epoch 5/20
219/219 [==============================] - 1s 6ms/step - loss: 2.3489 - accuracy: 0.4772
Epoch 6/20
219/219 [==============================] - 1s 7ms/step - loss: 2.2626 - accuracy: 0.5039
Epoch 7/20
219/219 [==============================] - 1s 6ms/step - loss: 2.2026 - accuracy: 0.5197
Epoch 8/20
219/219 [==============================] - 1s 6ms/step - loss: 2.1073 - accuracy: 0.5408
Epoch 9/20
219/219 [==============================] - 1s 6ms/step - loss: 2.0599 - accuracy: 0.5382
Epoch 10/20
219/219 [==============================] - 1s 6ms/step - loss: 2.0127 - accur

INFO:tensorflow:Assets written to: saved_models/m0/assets


INFO:tensorflow:Assets written to: saved_models/m0/assets


(None, 50)
Epoch 1/20
219/219 [==============================] - 4s 6ms/step - loss: 4.2318 - accuracy: 0.2366
Epoch 2/20
219/219 [==============================] - 1s 6ms/step - loss: 2.7963 - accuracy: 0.4437
Epoch 3/20
219/219 [==============================] - 1s 6ms/step - loss: 2.4006 - accuracy: 0.4679
Epoch 4/20
219/219 [==============================] - 1s 6ms/step - loss: 2.1455 - accuracy: 0.5266
Epoch 5/20
219/219 [==============================] - 1s 6ms/step - loss: 1.9829 - accuracy: 0.5566
Epoch 6/20
219/219 [==============================] - 1s 6ms/step - loss: 1.8502 - accuracy: 0.5795
Epoch 7/20
219/219 [==============================] - 1s 6ms/step - loss: 1.7105 - accuracy: 0.6047
Epoch 8/20
219/219 [==============================] - 1s 6ms/step - loss: 1.6728 - accuracy: 0.6167
Epoch 9/20
219/219 [==============================] - 1s 6ms/step - loss: 1.5972 - accuracy: 0.6283
Epoch 10/20
219/219 [==============================] - 1s 6ms/step - loss: 1.5064 - accur

INFO:tensorflow:Assets written to: saved_models/m1/assets


...

INFO:tensorflow:Assets written to: saved_models/m13/assets


INFO:tensorflow:Assets written to: saved_models/m13/assets


(None, 50)
Epoch 1/20
219/219 [==============================] - 4s 7ms/step - loss: 2.0626 - accuracy: 0.8130
Epoch 2/20
219/219 [==============================] - 2s 7ms/step - loss: 0.7943 - accuracy: 0.8467
Epoch 3/20
219/219 [==============================] - 2s 7ms/step - loss: 0.7071 - accuracy: 0.8630
Epoch 4/20
219/219 [==============================] - 2s 7ms/step - loss: 0.7168 - accuracy: 0.8541
Epoch 5/20
219/219 [==============================] - 2s 7ms/step - loss: 0.6528 - accuracy: 0.8664
Epoch 6/20
219/219 [==============================] - 2s 7ms/step - loss: 0.6320 - accuracy: 0.8721
Epoch 7/20
219/219 [==============================] - 2s 7ms/step - loss: 0.5970 - accuracy: 0.8777
Epoch 8/20
219/219 [==============================] - 2s 7ms/step - loss: 0.5549 - accuracy: 0.8853
Epoch 9/20
219/219 [==============================] - 2s 7ms/step - loss: 0.5502 - accuracy: 0.8878
Epoch 10/20
219/219 [==============================] - 2s 7ms/step - loss: 0.5180 - accur

INFO:tensorflow:Assets written to: saved_models/m14/assets


INFO:tensorflow:Assets written to: saved_models/m14/assets


In [67]:
for i in range(15):
  print(m[i].evaluate([np.array(inp[i][6995:]),np.array(inp[i][6995:])],np.array(first_arr[i][6995:])))

44/44 [==============================] - 1s 4ms/step - loss: 2.2516 - accuracy: 0.5983
[2.2515902519226074, 0.5982721447944641]
44/44 [==============================] - 1s 4ms/step - loss: 1.7168 - accuracy: 0.6717
[1.7168408632278442, 0.6717062592506409]
44/44 [==============================] - 1s 4ms/step - loss: 1.9706 - accuracy: 0.6631
[1.9706201553344727, 0.6630669832229614]
44/44 [==============================] - 1s 4ms/step - loss: 2.1461 - accuracy: 0.6631
[2.1461403369903564, 0.6630669832229614]
44/44 [==============================] - 1s 4ms/step - loss: 2.0983 - accuracy: 0.6508
[2.0983030796051025, 0.650827944278717]
44/44 [==============================] - 1s 4ms/step - loss: 1.9708 - accuracy: 0.6753
[1.9708073139190674, 0.675305962562561]
44/44 [==============================] - 1s 4ms/step - loss: 1.6366 - accuracy: 0.7379
[1.636561393737793, 0.7379409670829773]
44/44 [==============================] - 1s 4ms/step - loss: 1.4818 - accuracy: 0.7509
[1.4817876815795898,

In [68]:
import numpy as np
def normalise(k,v):
    if v=='dual' or v=='vga':
        return '2'
    if v=='yes':
        return '1'
    if v=='no':
        return '0'
    if isint(v)!=1 and isfloat(v)!=1:
        return v
    v=float(v) 
    if k=='battery':
        return v/(10000)
    elif k=='memory':
        return v/(64) 
    elif k=='camera':
        return v/40 
    elif k=='cost':
        return v/500
    elif k=='size':
        return (v-100)/100
    else:
        return v
def similarity(word1,word2):
    if word1 in word_to_emb:
        arr1=word_to_emb[word1] 
    else:
        arr1=[0]*50 
    if word2 in word_to_emb:
        arr2=word_to_emb[word2]
    else:
        arr2=[0]*50
    sm=0
    for i in range(50):
        sm+=abs(float(arr1[i])*100-float(arr2[i])*100) 
    return sm
def min_index(temp_arr,k,db):
    mn=1000000000
    mn_i
    for d in temp_arr:
        if int(db[d][k])<mn:
            mn_i=d
            mn=db[d][k]
    return mn_i,mn
color_similarity={'rest':2,'gold':0.14,'blue':0.5,'green':0.6,'red':0.7,'pink':0.35,'white':0.1,'black':0.9,'brown':0.8,'grey':0.75,'yellow':0.2,'silver':0.15}
brand_similarity={'rest':2,'asus':0.1,'acer':0.13,'samsung':0.7,'xiaomi':0.74,'nokia':0.64,'motorola':0.23,'apple':0.9,'huawei':0.85,'blackberry':0.8,'htc':0.83}
os_similarity={'rest':2,'and':0.2,'mic':0.5,'win':0.8}
def database_index(bspan_arr,db,deselect_arr):
    temp_change={}
    selected_arr=[] 
    Is_present=0
    for i in range(len(db)):
        if i not in deselect_arr:
            selected_arr.append(i) 
    for k,v in bspan_arr.items():
        #v=normalise(k,v)
        bspan=[k,v] 
        temp_arr=[]
        for d in selected_arr:
            if db[d][slot_to_key[bspan[0]]]==bspan[1]: 
                temp_arr.append(d) 
        if len(temp_arr)!=0:
            selected_arr=[] 
            for temp in temp_arr:
                selected_arr.append(temp)
        elif len(temp_arr)==0 and v not in ['high','good','less']:
            Is_present=-1 
#         elif k=='cost' and v=='less':
#             temp_change[k]=v
#             bspan_arr[k]=100 
#         elif k=='battery' and (v=='good' or v=='high'):
#             temp_change[k]=v
#             bspan_arr[k]=5000 
#         elif k=='size' and (v=='good' or v=='high'):
#             temp_change[k]=v
#             bspan_arr[k]=180 
#     for k,v in bspan_arr.items():
#         if v in ['more','good','high','better']:
#             if len(selected_arr)>8:
#                 temp_arr=[]
#                 for d in selected_arr:
#                     if len(temp_arr)<8:
#                         temp_arr.append(d) 
#                     else:
#                         temp_arr.sort() 
#                         if db[temp_arr[0]][k]<db[d][k]:
#                             temp_arr[0]=d
#             if len(temp_arr)!=0 and isint(v)==0:
#                 selected_arr=[] 
#                 for temp in temp_arr:
#                     selected_arr.append(temp)
    #print(selected_arr[:5],len(selected_arr))
    if Is_present!=-1:
        return selected_arr[0],Is_present
    mn=10000000
    min_index=0
    for d in selected_arr:
        similarity_value=0 
        for k,v in bspan_arr.items():
            #v=normalise(k,v)
            bspan=[k,v]
            if isint(v)==1 or isfloat(v)==1:
                similarity_value+=abs(float(normalise(k,v))-float(normalise(k,db[d][slot_to_key[k]])))
            if k=='color':
                color_name=db[d][slot_to_key[k]] 
                if color_name not in color_similarity:
                    color_name='rest'
                if v not in color_similarity:
                    v='rest'
                similarity_value+=abs(color_similarity[color_name]-color_similarity[v])
            if k=='brand':
                brand_name=db[d][slot_to_key[k]]
                if brand_name not in brand_similarity:
                    brand_name='rest' 
                if v not in brand_similarity:
                    v='rest'
                similarity_value+=abs(brand_similarity[brand_name]-brand_similarity[v]) 
            if k=='os':
                os_name=db[d][slot_to_key[k]]
                if os_name not in os_similarity:
                    os_name='rest'
                if v not in os_similarity:
                    v='rest' 
                similarity_value+=abs(os_similarity[os_name]-os_similarity[v]) 
            #similarity_value+=similarity(normalise(k,db[d][slot_to_key[bspan[0]]]),bspan[1]) 
            #print(bspan,d,similarity_value)
        if similarity_value<mn:
            mn=similarity_value 
            min_index=d 
            #print(min_index)
    #print(mn,db[min_index])
    for k in temp_change:
        bspan_arr[k]=temp_change[k]
    return min_index,Is_present 
# def new_select(slt,user_uttr,test_i):
#     slt_arr=get_pos(slt) 
#     inp_to_db=[] 
#     for i in range(12):
#         inp_to_db.append([0]*50) 
#     for slt_i in slt_arr:
#         if slot[slt_i] in key_to_pos:
#           val_arr=m1[slt_i].predict(np.array([user_uttr])) 
#           ind=np.argmax(val_arr[0]) 
#           ans=values_of_slot[slot[slt_i]][ind]
#           for i in range(len(user_uttr)):
#             if np.argmax(copy_model[slot[slt_i]].predict(np.array([user_uttr[i]]))[0])==1:
#               if user_arr[test_i][i] in word_to_emb:
#                 ans=user_arr[test_i][i] 
#               break
#           inp_to_db[key_to_pos[slot[slt_i]]]=word_to_emb[ans] 
#           print(slot[slt_i],ans) 
#     match=100000000
#     pos=0
#     for i in range(len(db_in)):
#         temp_match=similarity_match(inp_to_db,db_in[2183])
#         if temp_match<=match:
#             match=temp_match 
#             #print(match)
#             pos=i 
#     return db[pos]

    

In [69]:
def bspan_decode(dial,emb_dial,bspan_arr):
    dial_arr=padd(dial)
    #print(dial,dial_arr)
    slot_ot=m2.predict([np.array([emb_dial]),np.array([emb_dial])]) 
    slt_arr=get_pos(slot_ot[0]) 
    if 'want' in dial or 'need' in dial:
        for r in reqs:
            if r in dial:
                if r=='ram':
                    r='memory'
                if r=='dualsim':
                    r='sim'
                #print(r,mp_sl[r],slot[mp_sl[r]])
                slt_arr.append(mp_sl[r])
    #print(slt_arr) 
    for slt_i in slt_arr:
        if slot[slt_i] in key_to_pos:
          val_arr=m1[slt_i].predict([np.array([emb_dial]),np.array([emb_dial])]) 
          ind=np.argmax(val_arr[0]) 
          ans=values_of_slot[slot[slt_i]][ind]
          for i in range(len(emb_dial)):
            #print(slot[slt_i],dial_arr[i]) 
            if np.argmax(copy_model[slot[slt_i]].predict(np.array([emb_dial[i]]))[0])==1:
              if dial_arr[i] in word_to_emb:
                ans=dial_arr[i] 
              break 
          if slot[slt_i] in ['sim','os','brand'] and ans not in dial:
            pass
          else:
            bspan_arr[slot[slt_i]]=ans 
    if 'expensive' in dial and 'too' not in dial:
        bspan_arr['cost']='expensive'
    return bspan_arr

In [70]:
def find_best_feature(d):
    mx=0
    ret=''
    for k,v in db[d].items():
        if k not in ['review_person','year_name','month_name','ref_no'] and (isint(v)==1 or isfloat(v)==1):
            if normalise(k,v)>mx:
                mx=normalise(k,v) 
                ret=str(v)+' '+str(k) 
    return ret

In [71]:
def match_fun(arr1,arr2):
    reqs2=['cost','color','sim','year','month','ram','camera','display','battery','weight','os','rating','gps','month','model']
    t1,t2=[],[] 
    for a in arr1:
        if a in reqs2 and a not in t1:
            t1.append(a)
    for a in arr2:
        if a in reqs2 and a not in t2:
            t2.append(a)
    return t1==t2


In [72]:
dial_len=0
for i in range(800,1000):
    dial_len+=len(User[i])
print("average dialogue length=",dial_len/200)
dial_len=0 
for i in range(800):
    dial_len+=len(User[i]) 
print(dial_len) 

average dialogue length= 7.96
6792


In [75]:
from skimage import io
import matplotlib.pyplot as plt
import random
bleu_total=0
tot=0
match=0
slot_match=0
success_match=0
req_match=0
for i in range(800,820):
    bspan_arr={}
    deselect_arr=[]
    d_index=random.randint(0,2000)
    req_arr=[]
    match_bin=1
    s_bin=1
    req_bin=1
    pers=0
    for j in range(len(User[i])):
        query=User[i][j]
        #print(query)
        emb_dial=sent_to_emb(query)
        sentiment=np.argmax(sentiment_model.predict(np.array([emb_dial]))[0])-1
        if sentiment==-1:
            pers=1 
        elif sentiment==-1:
            pers=-1 
        else:
            pers=0
        if pers==1:
            deselect_arr.append(d_index) 
        request_span=request_model.predict([np.array([emb_dial]),np.array([emb_dial])])[0]
        request_span=ind_to_span[np.argmax(request_span)]
        true_span=ind_to_span[np.argmax(request_arr[dial_len+tot])][1] 
        if request_span[1]=='image' or 'image' in query:
            url=db[d_index]['img_url']
            image = io.imread(url)
            plt.imshow(image)
            plt.show()
        #print(out,request_span)
        bspan_mp=bspan_decode(query,emb_dial,bspan_arr) 
        bspan_arr={k:bspan_mp[k] for k in sorted(bspan_mp)}
        d_index,is_present=database_index(bspan_arr,db,deselect_arr)
        if is_present==-1:
            emb_dial.insert(0,[1]*50) 
        else:
            emb_dial.insert(0,[0]*50)
        out=beam_search(emb_dial,m,first,request_span,is_present,pers)
#         for k,v in slot_to_key.items():
#             if v not in out and k==Request[i][j]:
#                 match_bin=0
#         print(out)
#         print(bleu_score(out,Agent_Arr[tot]))
#         print(Agent_Arr[tot])
        if true_span not in ['null','rec_slt'] and slot_to_key[true_span] not in out:
            req_bin=0
        bleu_total+=bleu_score(out,Agent_Arr[dial_len+tot])
        for k,v in db[d_index].items():
            out=out.replace(k,str(v)) 
        out=out.replace('best_feature',find_best_feature(d_index))
        print(query,out,bspan_arr,request_span[1],"--- --- ---",sep='\n')
        tot+=1
#         if is_present==-1:
#             bspan_arr=update_bspan(bspan_arr,d_index)
    #print(bspan_arr)
    for j in range(len(Slot[i])):
        if Slot[i][j]!='null' and Value[i][j]!=db[d_index][slot_to_key[Slot[i][j]]] and Value[i][j] not in ['medium','good','less','high','light','latest','expensive','small','big','heavy']:
            s_bin=0
            print(Slot[i][j],Value[i][j],db[d_index][slot_to_key[Slot[i][j]]])
        if Slot[i][j]!='null' and (Slot[i][j] not in bspan_arr or Value[i][j]!=bspan_arr[Slot[i][j]]):
            match_bin=0
            print(Slot[i][j],Value[i][j],db[d_index][slot_to_key[Slot[i][j]]])
    slot_match+=match_bin
    success_match+=(s_bin and req_bin)
    req_match+=req_bin
    print("slot_match=",slot_match/(i-800+1))
    print("req_match=",req_match/(i-800+1)) 
    print("success_match=",success_match/(i-800+1))
#     print(bspan_arr)
#     for k in bspan_arr:
#         print(k,db[d_index][slot_to_key[k]],end=' || ')
#     print('')
    print(i,'=================================================================')
    print("bleu_total=",bleu_total/tot)
print("request_match=",req_match/50)
print("slot_match=",slot_match/50)
print("success_match=",success_match/50)
print("bleu_total=",bleu_total/tot)

goodmorning, show me good phone.
sure , any specific model are you looking for ?     
{}
null
--- --- ---
yes,it has, i like gold color with radio facility.
ok booking             
{'color': 'gold', 'radio': 'yes'}
rec_slt
--- --- ---
is it working on android os?
yes , it has .          
{'color': 'gold', 'os': 'and', 'radio': 'yes'}
null
--- --- ---


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


what is its cost?
its cost is 450 euro          
{'color': 'gold', 'os': 'and', 'radio': 'yes'}
cost
--- --- ---


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


do you have expensive phone with more benefits?
this phone is very very good .        
{'color': 'gold', 'cost': 'expensive', 'os': 'and', 'radio': 'yes'}
null
--- --- ---
i will not go for lower cost phone as i believe it don't provide good feautres.
then you should buy p8 energy pro by allview . it has very good phone . 
{'color': 'gold', 'cost': 'expensive', 'os': 'and', 'radio': 'yes'}
null
--- --- ---
what is its weight?
it is 209.8 gms           
{'color': 'gold', 'cost': 'expensive', 'os': 'and', 'radio': 'yes'}
weight
--- --- ---
okay, i like this phone. pack it for me.
good choice , i am booking it for you . your reference number is 162
{'color': 'gold', 'cost': 'expensive', 'os': 'and', 'radio': 'yes'}
null
--- --- ---
thanks for help.
welcome , it 's my pleasure         
{'color': 'gold', 'cost': 'expensive', 'os': 'and', 'radio': 'yes'}
null
--- --- ---
slot_match= 1.0
req_match= 0.0
success_match= 0.0
800 =================================================================
bl

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


i want to  a buy good phone.
sure , any specific features you are looking for ?     
{}
null
--- --- ---
i want high internal memory and light weight phone.
ok booking             
{'ram': 'good', 'weight': 'light'}
rec_slt
--- --- ---
what's its price?
it 's 170 euro           
{'ram': 'good', 'weight': 'light'}
cost
--- --- ---
it's too high for me.my budget is within 500euro
it is the most trending and very much this phone     
{'cost': 'less', 'ram': 'good', 'weight': 'light'}
null
--- --- ---
i can't afford it.
you can buy liquid z6 plus by acer .        
{'cost': 'less', 'ram': 'good', 'weight': 'light'}
null
--- --- ---
great,is it available in silver color?
yes , it has done          
{'color': 'silver', 'cost': 'less', 'ram': 'good', 'weight': 'light'}
null
--- --- ---
okay, i like this phone. pack it for me.
good choice , i am booking it for you . your reference number is 22
{'color': 'silver', 'cost': 'less', 'ram': 'good', 'weight': 'light'}
null
--- --- ---
thanks for help

In [ ]:
bleu_total=0
i=800
tot=0
for s in User[800:900]:
    bspan_arr={}
    j=0
    for dial in s:
        emb_dial=sent_to_emb(dial)
        request_span=request_model.predict([np.array([emb_dial]),np.array([emb_dial])])[0]
        request_span=ind_to_span[np.argmax(request_span)]
        out=beam_search(emb_dial,m,first,request_span) 
        #print(out,request_span)
        bspan_arr=bspan_decode(dial,emb_dial,bspan_arr) 
        d_index,is_present=database_index(bspan_arr,db)
        for k,v in db[d_index].items():
            out=out.replace(k,v) 
        bleu_total+=bleu_score(out,Agent[i][j])
        j+=1
        tot+=1
    temp_arr=[] 
    for sl in range(len(Slot[i])):
        if Slot[i][sl]!='null':
            temp_arr.append((Slot[i][sl],Value[i][sl]))
    print(temp_arr,bspan_arr,sep='\n')
    i+=1
    print('=================================================================')
print('bleu_score=',bleu_total/tot) 

In [ ]:
from skimage import io
import matplotlib.pyplot as plt
import random
if (1==1):
    bspan_arr={}
    deselect_arr=[]
    d_index=random.randint(0,2000)
    pers=0
    query=input()
    while(query!='quit'):
        emb_dial=sent_to_emb(query)
        sentiment=np.argmax(sentiment_model.predict(np.array([emb_dial]))[0])-1
        if pers==-1 and sentiment==-1:
            pers=1 
        elif sentiment==-1:
            pers=-1 
        else:
            pers=0
        #print(pers)
        if pers==1:
            deselect_arr.append(d_index) 
        request_span=request_model.predict([np.array([emb_dial]),np.array([emb_dial])])[0]
        request_span=ind_to_span[np.argmax(request_span)]
        if request_span[1]=='image' or 'image' in query:
            url=db[d_index]['img_url']
            image = io.imread(url)
            plt.imshow(image)
            plt.show()
        #print(out,request_span)
        bspan_mp=bspan_decode(query,emb_dial,bspan_arr) 
        bspan_arr={k:bspan_mp[k] for k in sorted(bspan_mp)}
        d_index,is_present=database_index(bspan_arr,db,deselect_arr)
        if is_present==-1:
            emb_dial.insert(0,[1]*50) 
        else:
            emb_dial.insert(0,[0]*50)
        out=beam_search(emb_dial,m,first,request_span,is_present,pers)
        for k,v in db[d_index].items():
            out=out.replace(k,v) 
        if is_present==-1:
            out='no we don\'t have this'+out+'it has best_feature'
        out=out.replace('best_feature',find_best_feature(d_index))
        print(out,bspan_arr,request_span[1],sep='\n')
        #print('obtained feature:',end=' ')
        print(pers)
        if is_present==-1:
            bspan_arr=update_bspan(bspan_arr,d_index)
        query=input()
#     temp_arr=[] 
#     for sl in range(len(Slot[i])):
#         if Slot[i][sl]!='null':
#             temp_arr.append((Slot[i][sl],Value[i][sl])) 
#     print(temp_arr,bspan_arr,sep='\n')
    for k in bspan_arr:
        print(k,db[d_index][slot_to_key[k]],end=' || ')
    print('')
    print('=================================================================')

In [ ]:
from skimage import io
import matplotlib.pyplot as plt
if (1==1):
    bspan_arr={}
    query=input()
    while(query!='quit'):
        emb_dial=sent_to_emb(query)
        request_span=request_model.predict([np.array([emb_dial]),np.array([emb_dial])])[0]
        request_span=ind_to_span[np.argmax(request_span)]
        if request_span[1]=='image' or 'image' in query:
            url=db[d_index]['img_url']
            image = io.imread(url)
            plt.imshow(image)
            plt.show()
        #print(out,request_span)
        bspan_arr=bspan_decode(query,emb_dial,bspan_arr) 
        d_index,is_present=database_index(bspan_arr,db,deselect_arr)
        if is_present==-1:
            emb_dial.insert(0,[1]*50) 
        else:
            emb_dial.insert(0,[0]*50)
        out=beam_search(emb_dial,m,first,request_span,is_present)
        for k,v in db[d_index].items():
            out=out.replace(k,v) 
        print(out,bspan_arr,request_span,sep='\n')
        print('obtained feature:',end=' ')
        for k in bspan_arr:
            print(k,db[d_index][slot_to_key[k]],end=' || ')
        print('')
        if is_present==-1:
            bspan_arr=update_bspan(bspan_arr,d_index)
        query=input()
#     temp_arr=[] 
#     for sl in range(len(Slot[i])):
#         if Slot[i][sl]!='null':
#             temp_arr.append((Slot[i][sl],Value[i][sl])) 
#     print(temp_arr,bspan_arr,sep='\n')
    print('=================================================================')

In [ ]:
# inp_span=[] 
# out_span=[]
# cnt=0
# for i in range(2517):
#     cnt+=1 
#     inp_span.append([])
#     for j in range(12):
#       inp_span[-1].append(db_in[i][j])
# for p in range(10):
#     for i in range(2517):
#       cnt+=1 
#       inp_span.append([])
#       for j in range(12):
#         if j==12-p:
#           inp_span[-1].append([0]*50) 
#         else:
#           inp_span[-1].append(db_in[i][j]) 
#     for q in range(p+1,11):
#         for i in range(2517):
#           cnt+=1
#           inp_span.append([]) 
#           for j in range(12):
#             if j==p or j==q:
#               inp_span[-1].append(db_in[i][j]) 
#             else:
#               inp_span[-1].append([0]*50) 
#         for i in range(2517):
#           cnt+=1
#           inp_span.append([]) 
#           for j in range(12):
#             if j==11-p or j==11-q:
#               inp_span[-1].append([0]*50) 
#             else:
#               inp_span[-1].append(db_in[i][j]) 
#         for r in range(q+1,12):
#             for i in range(2517):
#                 cnt+=1
#                 inp_span.append([])
#                 #out_span.append([]) 
#                 # for j in range(2517):
#                 #     cnt+=1
#                 #     if j==i:
#                 #         out_span[-1].append(1) 
#                 #     else:
#                 #         out_span[-1].append(0) 
#                 for j in range(12):
#                     if j==p or j==q or j==r:
#                         inp_span[-1].append(db_in[i][j]) 
#                     else:
#                         inp_span[-1].append([0]*50) 
                
# print(cnt)
# for i in range(150000):
#   out_span.append(db_out[i%2517]) 

In [ ]:
# out1=m2.predict(np.array(user_emb[2940:]))
# inp1=[] 
# for out in out1:
#   #print(out)
#   inp1.append([out])
# out2=m1.predict(np.array(inp1))
# count=0
# slct=0 
# slot_out=m2.predict(np.array(user_emb[6000:])) 
# for i in range(6000,8402): 
#   print(User_Arr[i])
#   slct=new_select(slot_out[i-6000],user_emb[i],i) 
#   sent=out[i-6000] 
#   for k in slct:
#     sent=sent.replace(k,str(slct[k]))  
#   count+=bleu_score(sent,Agent_Arr[i])
#   print(sent)
#   print('') 
# print(count/2402)

In [ ]:
# count=0
# slct=0
# #out=whole_beam_search(user_emb[6000:],m,first) 
# slot_out=m2.predict(np.array(user_emb[6000:])) 
# for i in range(6000,8402): 
#   print(User_Arr[i])
#   #slct=select(slot_out[i-2944],user_emb[i],slct,i) 
#   sent=out[i-6000] 
#   sent=sent.replace('<pad>','')
#   # for k in db[slct]:
#   #   sent=sent.replace(k,str(db[slct][k]))  
#   count+=bleu_score(sent,Agent_Arr[i])
#   print(slot_out[i-6000])
#   print(sent)
#   print('') 
# print(count/2402)

In [ ]:
# count=0
# slct=0
# tot=0
# #out=whole_beam_search(user_emb[:200],m,first) 
# slot_out=m2.predict(np.array(user_emb[6000:])) 
# for i in range(6000,8402): 
#   #print(User_Arr[i])
#   if slot[np.argmax(slot_out[i-6000])] in Slot_Arr[i]:
#     count+=1 
#   tot+=1
#   # slct=select(slot_out[i],user_emb[i],slct,i) 
#   # sent=out[i] 
#   # for k in db[slct]:
#   #   sent=sent.replace(k,str(db[slct][k]))  
#   # count+=bleu_score(sent,Agent_Arr[i])
#   # print(sent)
#   # print(Slot_Arr[i],Value_Arr[i]) 
#   # print('') 
# print(count/tot)

In [ ]:
# print(word_to_emb['samsung'])
# print(word_to_emb['asus']) 
# print(word_to_emb['nokia'])

In [ ]:
# first_emb_inp=[]
# for j in range(12):
#   first_emb_inp.append([])
#   for i in range(4201):
#     first_emb_inp[j].append([first_emb[j][i]])

In [ ]:
# temp_mp={}
# for arr in agent_arr:
#   for word in arr:
#     if word not in temp_mp:
#       temp_mp[word]=0 
# for word in temp_mp:
#   print(word) 

In [ ]:
# def similarity_match(arr1,arr2):
#     sm=0
#     for i in range(12):
#         if arr1[i]!=[0]*50:
#             for j in range(50):
#                 sm+=abs(float(arr1[i][j])*100-float(arr2[i][j])*100) 
#     return sm 
    
# temp_db=[]
# temp_db.append([0]*50)
# temp_db.append([0]*50)
# temp_db.append([0]*50)
# temp_db.append(word_to_emb['6']) 
# temp_db.append([0]*50)
# temp_db.append([0]*50)
# temp_db.append([0]*50)
# temp_db.append([0]*50)
# temp_db.append(word_to_emb['samsung']) 
# temp_db.append([0]*50) 
# temp_db.append(word_to_emb['200']) 
# temp_db.append([0]*50)
# match=100000000
# pos=0
# for i in range(len(db_in)):
#     temp_match=similarity_match(temp_db,db_in[2183])
#     if temp_match<=match:
#         match=temp_match 
#         #print(match)
#         pos=i 
# print(db[pos])
# sm=0
# for i in range(50):
#     sm+=abs(float(word_to_emb['samsung'][i])-float(word_to_emb['samsung'][i]))
# print(sm)   

In [ ]:
# out=m1.predict(np.array(user_emb[2940:])) 
# out_slot=m2.predict(np.array(user_emb[2940:])) 
# out_val=m2.predict(np.array(user_emb[2940:])) 
# n=0
# for i in range(200,1000):
#   q=(User_Arr[i+2940])
#   act=ug_action[pos(out[i])]
#   a=action_to_response[act]
#   slt=slot[pos(out_slot[i])] 
#   val=value[pos(out_val[i])]
#   arr=gen_proper_response(q,a,act,slt,val,n) 
#   a=arr[0] 
#   n=arr[1] 
#   #bleu_score(Agent_Arr[i+2940],a) 
#   print(q,a,sep='\n')
#   print('')

In [ ]:
# ott=m_temp.predict(np.array(inp_span[:2000]))
# for o in ott:
#   print(db[np.argmax(o)]['brand_name'])

In [ ]:
# import random 
# def gen_proper_response(q,res,act,slt,val,n): 
#   if act=='feedback' or act=='confirm':
#     if slt!='null':
#       res=slot_to_response[slt] 
#     else:
#       res=action_to_response['confirm'] 
#   for b in Brand_Arr:
#     if b in q:
#       res=action_to_response['confirm'] 
#       val=b
#       break
#   if 'model_name' in res and 'brand_name' in res:
#     if val in Brand_Arr:
#       for i in range(len(db)):
#         phone=db[i]
#         if phone['brand_name']==val:
#           n=i 
#           break
#     elif val in os_arr:
#       for i in range(len(db)):
#         phone=db[i] 
#         if phone['os_name']==val:
#           n=i 
#           break 
#     else:
#       n=random.randint(0,2000) 
#   for k in db[n].keys():
#     if k in res:
#       res=res.replace(k,str(db[n][k])) 
#   return (res,n)

In [ ]:
# slot_to_response={} 
# slot_to_responses={} 
# for u in Slot_Arr:
#   if u not in slot_to_responses:
#     slot_to_responses[u]={}
#     slot_to_response[u]=''
# for i in range(4201):
#   if Agent_Arr[i] not in slot_to_responses[Slot_Arr[i]]:
#     slot_to_responses[Slot_Arr[i]][Agent_Arr[i]]=1 
#   else:
#     slot_to_responses[Slot_Arr[i]][Agent_Arr[i]]+=1 
# for u in slot_to_response:
#   mx=0 
#   st='' 
#   for reply in slot_to_responses[u]:
#     if slot_to_responses[u][reply]>mx:
#       mx=slot_to_responses[u][reply] 
#       st=reply 
#   slot_to_response[u]=st    
# slot_to_response={'null': 'my pleasure', 'camera': 'its picture quality is very good . it has cam_val mp of camera', 'color': 'this model_nameas strong features but available in color_name only .', 'os': 'it has os_name operating system in it.', 'touch': 'sure , model_name by brand_name will be a good option for you . it is a sturdy phone also .', 'cost': 'cost_price money_unit is its price', 'memory': 'mem_val is good enough for daily use', 'price': 'model_name has a price of cost_val.', 'year': 'it was released in year_name', 'company': 'model_name by brand_name is available to us.', 'review': 'it is getting good response . it has recently come in the market so less have reviewed it . many customers also forgot to give the feedback . trust me you will not regret buying it', 'dual': 'sure , we have many dual sim mobiles in our store . is there any other specific features you are looking for ?', 'rating': 'it has been rated rating_val by review_person.', 'battery': 'bat_val mah is good enough to lasts for a day .', 'weight': 'its weight is weight_val. ', 'brand': 'model_name by brand_name will be good for you.', 'radio': 'rad_bin it has rad_bin radio', 'sim': 'yes it has dual sim .', 'ram': 'it have mem_size gb of memory .', 'size': 'size_val is its size', 'gps': 'model_name by brand_name meets your requirements .', 'nll': 'yes it works .', 'model': 'mode_name is its name', 'apple': 'model_name by brand_name released date is year_name'}

In [ ]:
# first_mp=[]
# num_to_first=[] 
# first_arr_mp=[]
# first_arr=[]
# count=[]
# for j in range(12):
#   first_mp.append({}) 
#   num_to_first.append([])
#   first_arr_mp.append({})
#   cnt=0  
#   for i in range(4201):
#     if agent_arr[i][j] not in first_mp[j]:
#       first_mp[j][agent_arr[i][j]]=cnt 
#       num_to_first[j].append(agent_arr[i][j])
#       cnt+=1
#   count.append(cnt) 
#   for k in first_mp[j]:
#     first_arr_mp[j][k]=[]
#     for i in range(cnt):
#       if i==first_mp[j][k]:
#         first_arr_mp[j][k].append(1)
#       else: 
#         first_arr_mp[j][k].append(0) 
#   first_arr.append([]) 
#   for i in range(4201):
#     first_arr[j].append(first_arr_mp[j][agent_arr[i][j]])
#   #print(len(first_arr[j][0]))
# print(count) 

In [ ]:
# def select(slots,values,pos):
#   if slot[mp_sl['null']]==1:
#     return pos 
#   pos_arr=[]
#   for i in range(len(slots)):
#     if slots[i]>0.1:
#       if slot[i]=='null':
#         return pos
#       key=slot_to_key[slot[i]] 
#       for j in len(db):
#         if db[j][key].lower()==values[value[i]]:
#           pos_arr.append(j) 
#       if values[value[i]] in ['high','good','expensive','more']:
#         mx=0 
#         mx_pos=0
#         for j in len(db):
#           if isfloat(db[j][key])==1 and float(db[j][key])>mx:
#             mx=float(db[j][key]) 
#             mx_pos=j 
#         pos_arr.append(mx_pos) 
#       if values[value[i]] in ['less','low','cheap','bad']:
#         mx=10000000 
#         mx_pos=0
#         for j in len(db):
#           if isfloat(db[j][key])==1 and float(db[j][key])<mx:
#             mx=float(db[j][key]) 
#             mx_pos=j 
#         pos_arr.append(mx_pos) 
#   temp_mp={} 
#   for p in pos:
#     if p not in temp_mp:
#       temp_mp[p]=1 
#     else:
#       temp_mp[p]+=1
#   pos=0 
#   mx=0
#   for i in temp_mp:
#     if temp_mp[i]>mx:
#       mx=temp_mp[i] 
#       pos=i 
#   return pos 

In [ ]:
# def select(slt,user_uttr,slct,test_i):
#   if 1==0:
#     return slct 
#   else:
#     slt_arr=get_pos(slt) 
#     inp_to_db=[] 
#     for i in range(12):
#       inp_to_db.append([0]*50) 
#   for slt_i in slt_arr:
#     if slot[slt_i] in key_to_pos:
#       val_arr=m1[slt_i].predict(np.array([user_uttr])) 
#       ind=np.argmax(val_arr[0]) 
#       ans=values_of_slot[slot[slt_i]][ind]
#       for i in range(len(user_uttr)):
#         if np.argmax(copy_model[slot[slt_i]].predict(np.array([user_uttr[i]]))[0])==1:
#           if user_arr[test_i][i] in word_to_emb:
#             ans=user_arr[test_i][i] 
#           break
#       inp_to_db[key_to_pos[slot[slt_i]]]=word_to_emb[ans] 
#       print(slot[slt_i],ans) 
#   ans_arr=new_model.predict(np.array([inp_to_db])) 
#   return np.argmax(ans_arr[0])

In [ ]:
# inp_encoder_np=np.array(inp_slot).reshape(9326,12,50) 
# out_encoder_np=np.array(inp_slot).reshape(9326,12,50)  
# slot_arr_np=np.array(out_slot).reshape(9326,25)
# slot_model,encoder_model,encoder_states=seq2seq(inp_encoder_np,out_encoder_np,slot_arr_np,25) 
# decoder_model=decoderModel(encoder_states,inp_encoder_np,slot_arr_np,25) 
# def model1(X,Y,input_size1,input_size2,output_size):
#   m=Sequential()
#   m.add(Bidirectional(LSTM(256,input_shape=(None,input_size2))))
#   #m.add(LSTM(128))
#   m.add(Dropout(0.5))
#   m.add(Dense(512,activation='relu'))
#   m.add(Dense(256,activation='relu'))
#   m.add(Dropout(0.5)) 
#   m.add(Dense(output_size,activation='softmax'))
#   m.compile('Adam','categorical_crossentropy',['accuracy'])
#   m.fit(X,Y,epochs=5)
#   return m 
# out=copy_model['color'].predict(np.array(sent_to_emb('black color')))
# print(out,User_Arr[1]) 
# m1.evaluate(np.array(inp_val[2940:]),np.array(out_val[2940:]))
# out=m1.predict(np.array(inp_val[2940:])) 
# print(get_pos(out[6]),get_pos(value_arr[2946]))
# # for i in range(len(out)):
# #   print(pos(out[i]),get_pos(slot_arr[i+2940])) 
# len(inp_slot)